## LangChain 사용해서 지원사업 공고 파일(pdf, hwp, png)을 요약해보자!

In [2]:
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 

- LangChain 모델이 참고할 비슷한 용어 사전
- 개발팀이 직접 정의
- (추후에 유사한 단어가 나오면 사전에 추가하는 기능도 구현할 예정)

In [ ]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {  
                        '사업자 유형' : ['개인사업자', '법인사업자', '창업(예비사업자 포함)', '재창업', '기존사업자'],
                        '지원업종' : ['자동차 및 부품 판매업','도매 및 상품 중개업','소매업(자동차 제외)','숙박업', '음식점업', '제조업','교육 서비스업', '협회 및 단체, 수리 및 기타 개인 서비스업','부동산업','전문, 과학 및 기술 서비스업','예술, 스포츠 및 여가관련 서비스업','정보통신업','농업, 임업 및 어업','건설업','운수 및 창고업','보건업 및 사회복지 서비스업','사업시설 관리, 사업 지원 및 임대 서비스업','금융 및 보험업','전기, 가스, 증기 및 공기 조절 공급업','광업','수도, 하수 및 폐기물 처리, 원료 재생업','가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동','공공행정, 국방 및 사회보장 행정','국제 및 외국기관'],
                        '정보통신업' : ['정보통신 서비스업', '정보통신기기 제조업', '소프트웨어 및 컴퓨터 관련 서비스업'],
                        '공고 특성' : ['청년 대상', '여성 대상', '소상공인', '대출', '마케팅·홍보', '보조금', '폐업', '고용지원', '시설·환경 개선', '입주·임대 지원', '희망리턴패키지', '고용유지지원금', '희망대출플러스', '두루누리지원금', '창업패키지', '노란우산공제', '이커머스 입점피해'],
                        '창업패키지' : ['예비창업패키지', '초기창업패키지', '청년창업사관학교'],
                        '신청기간' : ['신청 기간', '접수 기간', '자금 소진시까지'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상', '접수방법','접수'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '우대사항' : ['벤처기업', '여성기업', '사회적기업(인증)', '장애인기업', '이노비즈제도','메인비즈제도','가족친화인증기업', '수출유망중소기업','녹색기술인증기업','내일채움공제 가입기업','글로벌 강소기업', '기술혁신중소기업'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준', '지원 절차'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용', '지원조건'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의', '접수'],
                        '기타' : ['기타']
                        }

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [4]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [35]:
# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
file_path = f"금융/2025년 상반기 보증연계투자 대상기업 공모.hwp"

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('/')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('/')[1].split('.')[1]

# file_type이 pdf면 get_pdf_text, hwp면 get_hwp_text를 사용하여 텍스트 추출
if file_type == 'pdf':
    full_text = get_pdf_text(file_path)
elif file_type == 'hwp':
    full_text = get_hwp_text(file_path)

In [36]:
full_text

'\x02\x00\x00\x00\x00\x02\x02\x00\x00\x00\x00\x02\x0b\x00\x00\x00\x00\x0b\r\n2025년 상반기 보증연계투자 대상기업 공모\r\n기술보증기금(이하 기보)에서는 기술력과 사업성이 우수한 유망 창업기업에 대한 투자 활성화를 위해 다음과 같이 2025년 상반기 보증연계투자 대상기업을 공모하니, 많은 관심과 참여를 바랍니다.\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f실시 목적\r\n 기술력과 사업성이 우수하고 미래 성장이 기대되는 유망 창업기업의 육성을 위해 선제적으로 공적 투자를 지원\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f신청 자격\r\n 기술력, 사업성 및 성장잠재력 등을 갖춘 중소기업으로, 아래 기본요건을 모두 충족하는 창업기업\r\n\x0b\x00\x00\x00\x00\x0b\r\n\x02\x00\x00\x00\x00\x02\x1f법인설립 후 5년 이내의 기술혁신선도형기업*\r\n  * 단, R&D 및 녹색혁신성장미래전략산업 영위기업 등은 설립 후 5년 초과7년 이내 기업도 가능\r\n\x1f기보 보증기업 또는 보증승인(예정)기업\r\n\x1f최근 1년 이내 기보 기술사업평가등급이 BB등급 이상인 기업*\r\n  * 최근 1년 이내 평가등급이 없더라도 기보 영업점 담당자가 투자지원 가능성이 높다고 판단한 기업은 신청 가능\r\n\x1f보증금지기업 또는 보증제한기업에 해당되지 않는 기업\r\n\x1f벤처캐피탈, 은행 등 기관투자자로부터 투자를 받지 않은 기업*\r\n  * 단, 지방소재기업 및 모태출자조합으로부터 3억원 이하 투자를 받은 기업은 신청 가능\r\n\x1f보증연계투자(투자옵션부보증 포함) 잔액이 없는 기업*\r\n     위 , , ,  해당 여부는 기보 영업점 담당자에게 문의\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f공모 절차 및 지원 내용\r\n\x0b\x00\x00\x00\x00\x0b\r\n\x0

In [ ]:
'\x02\x00\x00\x00\x00\x02\x02\x00\x00\x00\x00\x02\x0b\x00\x00\x00\x00\x0b\r\n2025년 상반기 보증연계투자 대상기업 공모\r\n기술보증기금(이하 기보)에서는 기술력과 사업성이 우수한 유망 창업기업에 대한 투자 활성화를 위해 다음과 같이 2025년 상반기 보증연계투자 대상기업을 공모하니, 많은 관심과 참여를 바랍니다.\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f실시 목적\r\n 기술력과 사업성이 우수하고 미래 성장이 기대되는 유망 창업기업의 육성을 위해 선제적으로 공적 투자를 지원\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f신청 자격\r\n 기술력, 사업성 및 성장잠재력 등을 갖춘 중소기업으로, 아래 기본요건을 모두 충족하는 창업기업\r\n\x0b\x00\x00\x00\x00\x0b\r\n\x02\x00\x00\x00\x00\x02\x1f법인설립 후 5년 이내의 기술혁신선도형기업*\r\n  * 단, R&D 및 녹색혁신성장미래전략산업 영위기업 등은 설립 후 5년 초과7년 이내 기업도 가능\r\n\x1f기보 보증기업 또는 보증승인(예정)기업\r\n\x1f최근 1년 이내 기보 기술사업평가등급이 BB등급 이상인 기업*\r\n  * 최근 1년 이내 평가등급이 없더라도 기보 영업점 담당자가 투자지원 가능성이 높다고 판단한 기업은 신청 가능\r\n\x1f보증금지기업 또는 보증제한기업에 해당되지 않는 기업\r\n\x1f벤처캐피탈, 은행 등 기관투자자로부터 투자를 받지 않은 기업*\r\n  * 단, 지방소재기업 및 모태출자조합으로부터 3억원 이하 투자를 받은 기업은 신청 가능\r\n\x1f보증연계투자(투자옵션부보증 포함) 잔액이 없는 기업*\r\n     위 , , ,  해당 여부는 기보 영업점 담당자에게 문의\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f공모 절차 및 지원 내용\r\n\x0b\x00\x00\x00\x00\x0b\r\n\x02\x00\x00\x00\x00\x02 보증연계투자 대상기업 공모 절차 \r\n\x0b\x00\x00\x00\x00\x0b\r\n공모제\r\n시행공고\r\n\r\n공모추천\r\n\r\n예비검토\r\n\r\n기술평가,\r\n투자심사\r\n\r\n투자\r\n의사결정\r\n\r\n투자실행\r\n홈페이지 등\r\n상담/접수\r\n1차\x1f선별\r\n2차\x1f선별\r\n최종 결정\r\n계약체결 등\r\n벤처혁신\r\n금융부\r\n영업점*\r\n\x02\x00\x00\x00\x00\x02(콘텐츠)\r\n벤처투자\r\n금융센터\r\n\x02\x00\x00\x00\x00\x02(콘텐츠)\r\n벤처투자\r\n금융센터/\r\n기술혁신센터**\r\n\x02\x00\x00\x00\x00\x02(콘텐츠)\r\n벤처투자\r\n금융센터/\r\n투자심의회\r\n\x02\x00\x00\x00\x00\x02(콘텐츠)\r\n벤처투자\r\n금융센터\r\n * 중앙기술평가원, 기술혁신센터, 소셜벤처가치평가센터, 서울문화콘텐츠금융센터, 지식재산공제센터 등 포함\r\n** 투자심사용 기술평가 업무수행\r\n (공모기간) 25.\x1f3.\x1f17(월)  25.\x1f5.\x1f30(금)\r\n (접수방법) 기보 영업점 담당자와 상담 후 접수\r\n (투자지원 규모) 25년 상반기 400억원 내외\r\n (투자방식) 주식전환사채신주인수권부사채 인수, Convertible Note, SAFE 등\r\n (투자한도) 개별 기업당 30억원 이내\r\n\x0b\x00\x00\x00\x00\x0b\r\n\r\n\x1f기타 사항\r\n (유의사항) 예비검토 단계에서 기업 방문 또는 별도 통지 없이 투자 대상에서 제외될 수 있으며, 제출 서류는 반환하지 않음\r\n (문의처) 관할 영업점 또는 신청기업 본사사업장 인근 영업점\r\n     * (고객센터)\x1f\x1f15441120, 전국 영업점 연락처는 홈페이지(www.kibo.or.kr) 참조\r\n\x0b\x00\x00\x00\x00\x0b\r\n투자일정절차, 공모대상 해당 여부, 기술평가등급 및 접수 방법 등 공모 관련 자세한 사항은 귀사의 관할 영업점으로 문의 바랍니다.\r\n\n'

In [38]:
base_template = {f'''
📌 **{title} 요약 안내**

🤖 **사업 개요**

✅ **사업자 유형**

☑️ **지원업종**

💡 **공고특성**

👥 **지원대상**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**

⭐️ **우대사항**

👩 **여성 대상**

💁 **청년 대상**

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**

'''
}

# 📢 **기타 안내**  

In [21]:
explanation = {f'''
- 중소기업은 중소기업기본법에 정의된 기준을 따르며 법인사업자, 기존사업자를 포함해주세요.
- 시설·환경 개선 자금은 기업 운영에 필요한 기계설비, 공장설립, 토지구입 등에 사용되는 자금, 원자재 구매, 생산, 판매 등 기업의 운영에 필요한 자금입니다.
- 청년은 만 19세 이상 39세 이하를 의미합니다. 
- 전용 대상은 특정 대상만 신청이 가능한 경우를 의미하고, 우대 조건은 특정 대상에게 가점을 부여하는 경우를 의미합니다.
- 공고에서 언급된 모든 금리 우대 항목을 추출해주세요. 
- 여성대상과 청년대상은 대상이 여성만 또는 청년만 되는 내용이 있는 경우에 해당하며 O,X로 구분해주세요.
- 만약 '변동없음'이라고 기재되어 있다면, 기존에 제공된 값을 찾아서 사용하세요. 즉, 당초(1,3,5,6,9월)이라면 기존 값을 그대로 추출하세요.
'''}

#### LangChain 모델로 간단하게 요약해보기

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 정부 지원 사업 전문 분석가입니다. 

사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 마크다운 형식으로 작성해주세요

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}
참고 : {explanation}
---------------------------------------------------
사용자 요구 파일 : {text}

각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
"""

# 파일 요약을 위한 프롬프트 템플릿을 정의
prompt = PromptTemplate.from_template(template)

# OpenAI의 GPT-4o 모델을 사용하여 채팅 모델을 생성
model = ChatOpenAI(
    model='gpt-4o',  # 사용할 언어 모델 지정
    api_key=MY_API_KEY,  # OpenAI API 키 입력
    temperature=0
)

# 모델의 출력을 문자열로 변환하는 출력 파서
output_parser = StrOutputParser()

# 체인(Chain) 구성: 프롬프트 → 모델 → 출력 파서 순서로 연결
chain = prompt | model | output_parser

In [30]:
# template = """
# 각 자금별 제출서류를 원문 그대로 추출해주세요. 표는 마크다운 형식으로 추출해주세요.
# 사용자 요구 파일 : {text}
# """

# 파일 요약을 위한 프롬프트 템플릿을 정의
prompt = PromptTemplate.from_template(template)

# OpenAI의 GPT-4o 모델을 사용하여 채팅 모델을 생성
model = ChatOpenAI(
    model='gpt-4o',  # 사용할 언어 모델 지정
    api_key=MY_API_KEY,  # OpenAI API 키 입력
    temperature=0
)

# 모델의 출력을 문자열로 변환하는 출력 파서
output_parser = StrOutputParser()

# 체인(Chain) 구성: 프롬프트 → 모델 → 출력 파서 순서로 연결
chain = prompt | model | output_parser

In [37]:
title

'2025년 상반기 보증연계투자 대상기업 공모'

In [39]:
print(chain.invoke({'base_template':base_template,
                    'keyword_dictionary':keyword_dictionary,
                    'explanation': explanation,
                    'text':full_text}))

```
📌 **2025년 상반기 보증연계투자 대상기업 공모 요약 안내**

🤖 **사업 개요**

✅ **사업자 유형**
법인사업자, 창업(예비사업자 포함)

☑️ **지원업종**
전문, 과학 및 기술 서비스업

💡 **공고특성**
보조금

👥 **지원대상**
기술력, 사업성 및 성장잠재력 등을 갖춘 중소기업으로, 아래 기본요건을 모두 충족하는 창업기업
- 법인설립 후 5년 이내의 기술혁신선도형기업
  * 단, R&D 및 녹색혁신성장미래전략산업 영위기업 등은 설립 후 5년 초과 7년 이내 기업도 가능
- 기보 보증기업 또는 보증승인(예정)기업
- 최근 1년 이내 기보 기술사업평가등급이 BB등급 이상인 기업
  * 최근 1년 이내 평가등급이 없더라도 기보 영업점 담당자가 투자지원 가능성이 높다고 판단한 기업은 신청 가능
- 보증금지기업 또는 보증제한기업에 해당되지 않는 기업
- 벤처캐피탈, 은행 등 기관투자자로부터 투자를 받지 않은 기업
  * 단, 지방소재기업 및 모태출자조합으로부터 3억원 이하 투자를 받은 기업은 신청 가능
- 보증연계투자(투자옵션부보증 포함) 잔액이 없는 기업

📅 **신청기간**
2025.03.17(월) - 2025.05.30(금)

📝 **신청방법**
기보 영업점 담당자와 상담 후 접수

🚫 **지원제외 대상**
보증금지기업 또는 보증제한기업

⭐️ **우대사항**
기술혁신중소기업

👩 **여성 대상**
X

💁 **청년 대상**
X

⚠ **유의사항**
예비검토 단계에서 기업 방문 또는 별도 통지 없이 투자 대상에서 제외될 수 있으며, 제출 서류는 반환하지 않음

📑 **제출서류**
공고문 참조

📌 **선정절차**
| 단계          | 내용                                      |
|---------------|-------------------------------------------|
| 공모제        | 시행공고                                  |
| 공모추천

In [34]:
import pandas as pd

# 1인 자영업자 사회보험료 지원사업 제출서류
one_person_self_employed_documents = {
    "구분": ["필수제출서류", "", "", "선택제출서류", ""],
    "제출서류": [
        "사업신청서(행정정보공동이용동의서포함)[서식1]",
        "중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]",
        "통장사본(스마트폰캡처본도인정)",
        "사업자등록증또는사업자등록증명원사본1부.",
        "건강보험자격득실확인서사본1부.(신청일로부터1개월이내발급본)"
    ],
    "비고": ["", "", "수수료없음", "", "* 국민건강보험공단홈페이지(www.nhis.go.kr) 또는정부24에서발급가능행정정보공동이용미동의시별도제출"]
}

# 소상공인 사회보험료 지원사업 제출서류
small_business_documents = {
    "구분": ["신규신청", "신규신청", "신규신청", "신규신청", "신규신청", "신규신청", "신규신청", "변경신청", "변경신청", "변경신청"],
    "신청주기": ["최초1회", "최초1회", "최초1회", "최초1회", "최초1회", "최초1회", "최초1회", "근로자추가시(추가신청자만해당)", "근로자추가시(추가신청자만해당)", "근로자추가시(추가신청자만해당)"],
    "제출서류": [
        "소상공인고용·산재보험료지원신청서[서식2]",
        "사회보험료지원체크리스트[서식3]",
        "중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]",
        "개인정보수집·활용동의서(근로자용)[첨부]/보험료지원신청근로자전체",
        "사업자등록증, 사업자등록증명원, 고유번호증(비영리단체) 사본",
        "사업주또는사업장명의통장사본(스마트폰캡처본도인정)",
        "4대보험사업장가입자명부",
        "소상공인고용·산재보험료지원신청서[서식2]",
        "사회보험료지원체크리스트[서식3]",
        "개인정보수집·활용동의서(근로자용)[첨부]/변경신청근로자전체"
    ]
}

# Create DataFrames
df_one_person = pd.DataFrame(one_person_self_employed_documents)
df_small_business = pd.DataFrame(small_business_documents)

# Display DataFrames
print("1인 자영업자 사회보험료 지원사업 제출서류")
print(df_one_person)
print("\n소상공인 사회보험료 지원사업 제출서류")
print(df_small_business)

1인 자영업자 사회보험료 지원사업 제출서류
       구분                              제출서류  \
0  필수제출서류         사업신청서(행정정보공동이용동의서포함)[서식1]   
1           중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]   
2                          통장사본(스마트폰캡처본도인정)   
3  선택제출서류             사업자등록증또는사업자등록증명원사본1부.   
4          건강보험자격득실확인서사본1부.(신청일로부터1개월이내발급본)   

                                                  비고  
0                                                     
1                                                     
2                                              수수료없음  
3                                                     
4  * 국민건강보험공단홈페이지(www.nhis.go.kr) 또는정부24에서발급가능행정정...  

소상공인 사회보험료 지원사업 제출서류
     구분              신청주기                                 제출서류
0  신규신청              최초1회               소상공인고용·산재보험료지원신청서[서식2]
1  신규신청              최초1회                    사회보험료지원체크리스트[서식3]
2  신규신청              최초1회      중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]
3  신규신청              최초1회  개인정보수집·활용동의서(근로자용)[첨부]/보험료지원신청근로자전체
4  신규신청              최초1회 

## NER을 활용해보자
- 개체명 인식(Named Entity Recognition)이란 말 그대로 이름을 가진 개체(named entity)를 인식하겠다는 것을 의미
- NER(Named Entity Recognition)을 활용하면 비슷한 문맥을 가진 내용을 같은 범주로 분류하는 데 있어서 단어 사전 기반 방식보다 더 유연하고 강력하게 처리할 수 있음
- 단어 사전 방식은 특정 단어나 구문에 의존하기 때문에 문맥을 고려하지 못하는 한계가 있지만, NER은 문맥 정보를 학습하여 더 정확한 분류가 가능

In [20]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [21]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset

# 값 목록 정의
category_value = {
    '사업자 유형' : ['개인사업자', '법인사업자','창업(예비사업자 포함)', '재창업', '기존사업자'],
    '지원업종' : ['자동차 및 부품 판매업','도매 및 상품 중개업','소매업(자동차 제외)','숙박업', '음식점업', '제조업','교육 서비스업', '협회 및 단체, 수리 및 기타 개인 서비스업','부동산업','전문, 과학 및 기술 서비스업','예술, 스포츠 및 여가관련 서비스업','정보통신업','농업, 임업 및 어업','건설업','운수 및 창고업','보건업 및 사회복지 서비스업','사업시설 관리, 사업 지원 및 임대 서비스업','금융 및 보험업','전기, 가스, 증기 및 공기 조절 공급업','광업','수도, 하수 및 폐기물 처리, 원료 재생업','가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동','공공행정, 국방 및 사회보장 행정','국제 및 외국기관'],
    '정보통신업' : ['정보통신 서비스업', '정보통신기기 제조업', '소프트웨어 및 컴퓨터 관련 서비스업'],
    '공고 특성' : ['청년 대상', '여성 대상', '소상공인', '대출', '마케팅·홍보', '보조금', '폐업', '고용지원', '시설·환경 개선', '입주·임대 지원', '희망리턴패키지', '고용유지지원금', '희망대출플러스', '두루누리지원금', '창업패키지', '노란우산공제', '이커머스 입점피해'],
    '창업패키지' : ['예비창업패키지', '초기창업패키지', '청년창업사관학교'],
    '신청기간' : ['신청 기간', '접수 기간', '자금 소진시까지'],
    '신청방법' : ['지원 대상', '사업 대상', '융자 대상', '접수방법','접수'],
    '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
    '우대사항' : ['벤처기업', '여성기업', '사회적기업(인증)', '장애인기업', '이노비즈제도','메인비즈제도','가족친화인증기업', '수출유망중소기업','녹색기술인증기업','내일채움공제 가입기업','글로벌 강소기업', '기술혁신중소기업'],
    '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
    '제출 서류' : ['필요 서류', '서류 제출 목록'],
    '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준', '지원 절차'],
    '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용', '지원조건'],
    '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의', '접수'],
    '기타' : ['기타']
}

# 학습 데이터셋
train_data = [
  {"text": "부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다.", "labels":[(0, 48, "사업개요")]},
  {"text": "보증대상 : 부천시 소재 소상공인", "labels":[(7, 18, "사업자 유형")]},
  {"text": "대상 : 소상공인 특례보증 대상자", "labels": [(7, 17, "사업자 유형")]},
  {"text": "보증대상 : 부천시 소재 소상공인 ▸사업자 등록일로부터 2개월 이상 경과 된 사업자", "labels": [(7, 60, "지원대상")]},
  {"text": "대상 : 소상공인 특례보증 대상자", "labels": [(7, 17, "지원대상")]},
  {"text": "접수기간․ 상반기 : 2025. 1. 23. ~ 자금소진시까지 하반기 : 2025. 7. 1. ~ 자금소진시까지", "labels": [(0, 71, "신청기간")]},
  {"text": "특례보증 신청 및 접수 - 접수기간․ 상반기 : 2025. 1. 23. ~ 자금소진시까지․ 하반기 : 2025. 7. 1. ~ 자금소진시까지 - 접 수 처 : 경기신용보증재단 부천지점․ 연락처 : 1577-5900․ 주  소 : 부천시 원미구 소향로 217, 4층", "labels": [(0, 162, "신청방법")]},
  {"text": "특례보증 제외대상      - 지방세, 세외수입 등 체납이 있는 사업자       - 제한업종(금융업 ․ 유흥업 ․ 불법도박 및 사치 ․ 향락업종 등)", "labels": [(0, 93, "지원제외 대상")]},
  {"text": "▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수      ▸기존 보증금액 포함 5,000만원 이내 ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황), 재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정", "labels": [(0, 221, "유의사항")]},
  {"text": "준비서류 : 사업자등록증 사본, 매출증빙서류, 신분증 등", "labels": [(0, 32, "제출서류")]},
  {"text": "특례보증 처리 절차  상담․접수➡ 심사(서류․현장)➡보증서발급➡ 자금대출", "labels": [(0, 47, "선정절차")]},
  {"text": "1. 사 업 명 : 2025년 소상공인 특례보증 지원 보증한도 : 사업자별 최대 5,000만 원 ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수      ▸기존 보증금액 포함 5,000만원 이내      ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황),   재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정 2. 사 업 명 : 2025년 소상공인 이차보전 지원 내용 : 특례보증과 연계하여 대출이자 연 2% 1년간 지원      ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수    다. 협약은행 : 6개 관내 금융기관      ▸농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행", "labels": [(0, 394, "지원내용")]},
  {"text": "경기신용보증재단 부천지점(☎ 1577-5900)    또는 부천시 지역경제과(☎ 032-625-2766)", "labels": [(0, 60, "신청방법")]},
  {"text": "소상공인", "labels": [(0, 4, "지원업종")]},
  {"text": "특례보증", "labels":[(0, 4, "공고 특성")]},
]

# 데이터셋 변환
def convert_to_dataset(data):
    texts = [item['text'] for item in data]
    labels = [item['labels'] for item in data]
    return Dataset.from_dict({'text': texts, 'labels': labels})

train_dataset = convert_to_dataset(train_data)

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base-ner")
model = AutoModelForTokenClassification.from_pretrained("klue/bert-base-ner", num_labels=len(unique_labels)) # unique_labels는 라벨 목록

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    # ... 기타 설정
)

# Trainer 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

trainer.train()

# 예측 및 후처리
# ... 모델 예측 후 category_values를 사용하여 결과 필터링

ArrowInvalid: Could not convert '사업개요' with type str: tried to convert to int64

In [ ]:
"""
- 지원업종이 소상공인 등으로 특정 업종이 명시되어 있지 않은 경우, {category_value} 내의 모든 지원업종을 출력해주세요.
- 사업자 업종 중 '개인사업자', '법인사업자' 중 하나는 반드시 포함되어야 합니다.
- 사업자 업종 중 '창업(예비사업자 포함)', '재창업', '기존사업자' 중 하나는 반드시 포함되어야 합니다.

"""

In [ ]:
import openai
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset

# 값 목록 정의
category_value = {
    '사업자 유형': ['개인사업자', '법인사업자', '창업(예비사업자 포함)', '재창업', '기존사업자'],
    '지원업종': ['자동차 및 부품 판매업', '도매 및 상품 중개업', '소매업(자동차 제외)', '숙박업',
               '음식점업', '제조업', '교육 서비스업', '협회 및 단체, 수리 및 기타 개인 서비스업',
               '부동산업', '전문, 과학 및 기술 서비스업', '예술, 스포츠 및 여가관련 서비스업',
               '정보통신업', '농업, 임업 및 어업', '건설업', '운수 및 창고업', '보건업 및 사회복지 서비스업',
               '사업시설 관리, 사업 지원 및 임대 서비스업', '금융 및 보험업', '전기, 가스, 증기 및 공기 조절 공급업',
               '광업', '수도, 하수 및 폐기물 처리, 원료 재생업', '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동',
               '공공행정, 국방 및 사회보장 행정', '국제 및 외국기관'],
    '정보통신업': ['정보통신 서비스업', '정보통신기기 제조업', '소프트웨어 및 컴퓨터 관련 서비스업'],
    '공고 특성': ['청년 대상', '여성 대상', '소상공인', '대출', '마케팅·홍보', '보조금', '폐업', '고용지원',
               '시설·환경 개선', '입주·임대 지원', '희망리턴패키지', '고용유지지원금', '희망대출플러스',
               '두루누리지원금', '창업패키지', '노란우산공제', '이커머스 입점피해'],
    '창업패키지': ['예비창업패키지', '초기창업패키지', '청년창업사관학교'],
    '신청기간': ['신청 기간', '접수 기간', '자금 소진시까지'],
    '신청방법': ['지원 대상', '사업 대상', '융자 대상', '접수방법', '접수'],
    '지원제외': ['지원 제외', '지원 취소', '제외 대상', '제한 대상'],
    '우대사항': ['벤처기업', '여성기업', '사회적기업(인증)', '장애인기업', '이노비즈제도', '메인비즈제도',
               '가족친화인증기업', '수출유망중소기업', '녹색기술인증기업', '내일채움공제 가입기업', '글로벌 강소기업',
               '기술혁신중소기업'],
    '유의사항': ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
    '제출 서류': ['필요 서류', '서류 제출 목록'],
    '선정 절차': ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준', '지원 절차'],
    '지원 내용': ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용', '지원조건'],
    '문의처': ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의', '접수'],
    '기타': ['기타'],
    '사업개요': ['사업개요']
}

# 학습 데이터셋
train_data = [
    {"text": "부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다.",
     "labels": [(0, 48, "사업개요")]},
    {"text": "보증대상 : 부천시 소재 소상공인", "labels": [(7, 18, "소상공인")]},  # 사업자 유형이 아닌 공고 특성으로 판단
    {"text": "대상 : 소상공인 특례보증 대상자", "labels": [(7, 17, "소상공인")]},  # 사업자 유형이 아닌 공고 특성으로 판단
    {"text": "보증대상 : 부천시 소재 소상공인 ▸사업자 등록일로부터 2개월 이상 경과 된 사업자",
     "labels": [(7, 60, "지원대상")]},
    {"text": "대상 : 소상공인 특례보증 대상자", "labels": [(7, 17, "소상공인")]},  # 사업자 유형이 아닌 공고 특성으로 판단
    {"text": "접수기간․ 상반기 : 2025. 1. 23. ~ 자금소진시까지 하반기 : 2025. 7. 1. ~ 자금소진시까지",
     "labels": [(0, 71, "신청기간")]},
    {"text": "특례보증 신청 및 접수 - 접수기간․ 상반기 : 2025. 1. 23. ~ 자금소진시까지․ 하반기 : 2025. 7. 1. ~ 자금소진시까지 - 접 수 처 : 경기신용보증재단 부천지점․ 연락처 : 1577-5900․ 주  소 : 부천시 원미구 소향로 217, 4층",
     "labels": [(0, 162, "신청방법")]},
    {"text": "특례보증 제외대상      - 지방세, 세외수입 등 체납이 있는 사업자       - 제한업종(금융업 ․ 유흥업 ․ 불법도박 및 사치 ․ 향락업종 등)",
     "labels": [(0, 93, "지원제외")]},
    {"text": "▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수      ▸기존 보증금액 포함 5,000만원 이내 ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황), 재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정",
     "labels": [(0, 221, "유의사항")]},
    {"text": "준비서류 : 사업자등록증 사본, 매출증빙서류, 신분증 등", "labels": [(0, 32, "제출 서류")]},
    {"text": "특례보증 처리 절차  상담․접수➡ 심사(서류․현장)➡보증서발급➡ 자금대출",
     "labels": [(0, 47, "선정 절차")]},
    {"text": "1. 사 업 명 : 2025년 소상공인 특례보증 지원 보증한도 : 사업자별 최대 5,000만 원 ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수      ▸기존 보증금액 포함 5,000만원 이내      ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황),   재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정 2. 사 업 명 : 2025년 소상공인 이차보전 지원 내용 : 특례보증과 연계하여 대출이자 연 2% 1년간 지원      ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수    다. 협약은행 : 6개 관내 금융기관      ▸농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행",
     "labels": [(0, 394, "지원내용")]},
    {"text": "경기신용보증재단 부천지점(☎ 1577-5900)    또는 부천시 지역경제과(☎ 032-625-2766)",
     "labels": [(0, 60, "신청방법")]},
    {"text": "소상공인", "labels": [(0, 4, "지원업종")]},
    {"text": "특례보증", "labels": [(0, 4, "공고 특성")]},
]

# 라벨 목록 추출 및 고유 라벨 생성
unique_labels = sorted(list(set(label for item in train_data for _, _, label in item['labels'])))
unique_labels = ['O'] + unique_labels # O 라벨 추가

# 라벨-ID 매핑 딕셔너리 생성
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for label , i in label_to_id.items()}

def convert_to_dataset(data):
    texts = [item['text'] for item in data]
    labels = []
    for item in data:
        label_ids = []
        for start, end, label in item['labels']:
            label_ids.append((start, end, label_to_id[label]))  # 라벨을 ID로 변환
        labels.append(label_ids)
    return Dataset.from_dict({'text': texts, 'labels': labels})

train_dataset = convert_to_dataset(train_data)

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base-ner")
model = AutoModelForTokenClassification.from_pretrained("klue/bert-base-ner",
                                                      num_labels=len(unique_labels))  # unique_labels 사용

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    # ... 기타 설정 (필요에 따라 추가)
)

# Trainer 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

trainer.train()

# 모델 활용 함수
def extract_category_values(text, model, tokenizer, label_to_id, category_value):
    """
    학습된 모델을 사용하여 새로운 공고문 파일에서 category_value 추출
    """
    # 1. 토큰화 및 모델 예측
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1).squeeze().tolist()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())

    # 2. 예측 결과 디코딩 및 카테고리별 값 추출
    extracted_values = {category: [] for category in category_value.keys()}
    current_category = None
    current_entity = ""

    for token, prediction_id in zip(tokens, predictions):
        label = unique_labels[prediction_id]

        if label != "O":
            if current_category is None:
                current_category = label
                current_entity = token
            elif label == current_category:
                current_entity += token.replace("##", "")
            else:
                extracted_values[current_category].append(current_entity)
                current_category = label
                current_entity = token

        elif current_category is not None:
            extracted_values[current_category].append(current_entity)
            current_category = None
            current_entity = ""

    if current_category is not None:
        extracted_values[current_category].append(current_entity)

    # 3. 추출된 값을 category_value 내에서 선택 (예시)
    final_result = {}
    for category, extracted_list in extracted_values.items():
        if extracted_list:
            final_result[category] = []
            for extracted in extracted_list:
                # category_value 내에서 일치하는 값 찾기
                for value in category_value[category]:
                  # 필요에 따라 정확한 일치, 부분 일치, 유사도 비교 등을 수행
                  if value in extracted: 
                     final_result[category].append(value)
    return final_result

# OpenAI API 키 설정
openai.api_key = "YOUR_OPENAI_API_KEY"

def get_gpt_response(prompt):
    """GPT API 호출 및 답변 반환"""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # 또는 다른 모델 선택
        messages=[{"role": "user", "content": prompt}],
        temperature = 0.3 # 추가적인 파라미터 값 설정 필요에 따라 조절할수 있음.
    )
    return response["choices"][0]["message"]["content"].strip()

# 학습 데이터 요약 함수 (예시)
def summarize_train_data(train_data, category_value):
    """학습 데이터를 요약하여 자연어 텍스트로 변환"""

    summary = "다음은 공고문에서 추출해야 할 항목과 예시입니다.\n\n"
    for category, values in category_value.items():
        summary += f"{category}: {', '.join(values)}\n"

    example_data = "\n다음은 공고문 데이터의 몇 가지 예시입니다.\n\n"
    for item in train_data[:3]: # 몇개정도 예시를 추가할지 조정
        example_data += f"텍스트: {item['text']}\n"
        example_data += f"추출된 정보: {item['labels']}\n\n"

    return summary+example_data

def extract_values_from_text(text, train_data, category_value, base_template):
    """주어진 텍스트에서 학습된 데이터를 활용하여 GPT API로 값을 추출하고 템플릿에 맞춰 반환"""
    
    # 학습 데이터 요약
    train_data_summary = summarize_train_data(train_data, category_value)

    # GPT 프롬프트 구성
    prompt = f"{train_data_summary}\n\n주어진 정보를 바탕으로 다음 공고문에서 템플릿에 맞춰 정보를 추출해주세요.\n\n공고문: {text}\n\n출력 형식:\n{base_template}"

    # GPT API 호출 및 결과 활용
    gpt_response = get_gpt_response(prompt)
    return gpt_response

# 새로운 공고문 파일 예시
new_text = '부천시공고제2025-191호2025년 소상공인 특례보증․이차보전 지원계획 공고  부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다. 2025년  1월 23일                                             부  천  시  장1. 사 업 명 : 2025년 소상공인 특례보증 지원    가. 출 연 금 : 12억 원    나. 보증규모 : 120억 원       ※상반기 60억원 / 하반기 60억원 분리운영    다. 보증대상 : 부천시 소재 소상공인      ▸사업자 등록일로부터 2개월 이상 경과 된 사업자    라. 보증한도 : 사업자별 최대 5,000만 원      ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수      ▸기존 보증금액 포함 5,000만원 이내      ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황),   재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정     마. 특례보증 제외대상      - 지방세, 세외수입 등 체납이 있는 사업자       - 제한업종(금융업 ․ 유흥업 ․ 불법도박 및 사치 ․ 향락업종 등)\n    바. 특례보증 신청 및 접수      - 접수기간         ․ 상반기 : 2025. 1. 23. ~ 자금소진시까지        ․ 하반기 : 2025. 7. 1. ~ 자금소진시까지      - 접 수 처 : 경기신용보증재단 부천지점        ․ 연락처 : 1577-5900        ․ 주  소 : 부천시 원미구 소향로 217, 4층      - 준비서류 : 사업자등록증 사본, 매출증빙서류, 신분증 등    사. 특례보증 처리 절차  상담․접수➡ 심사(서류․현장)➡보증서발급➡ 자금대출경기신용보증재단경기신용보증재단경기신용보증재단은행2. 사 업 명 : 2025년 소상공인 이차보전 지원    가. 대상 : 소상공인 특례보증 대상자    나. 내용 : 특례보증과 연계하여 대출이자 연 2% 1년간 지원      ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수    다. 협약은행 : 6개 관내 금융기관      ▸농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행 ○ 기타 자세한 문의는 경기신용보증재단 부천지점(☎ 1577-5900)    또는 부천시 지역경제과(☎ 032-625-2766)로 문의하시기 바랍니다.'

base_template = '''
📌 **{title} 요약 안내**

✅ **사업자 유형**

☑️ **지원업종**

💡 **공고특성**

👥 **지원대상**

📅 **신청기간** 📝 **신청방법** 🚫 **지원제외 대상**

⭐️ **우대사항**

👩 **여성 대상**

💁 **청년 대상**

⚠ **유의사항** 📑 **제출서류** 📌 **선정절차** 💰 **지원내용** 📞 **문의처**
'''

# 추출 함수 사용 예시
result = extract_values_from_text(new_text, train_data, category_value, base_template)
print(result)

OSError: klue/bert-base-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

## Fine tuning 해서 성능 비교해보자

In [ ]:
from openai import OpenAI
from getpass import getpass
from datetime import datetime
from collections import defaultdict # 딕셔너리 디폴트 값을 지정해주는 모듈
import numpy as np
import time
import json
import tiktoken # openai 모델들이 사용하는 base 토큰화+인코등 모듈

In [86]:
# Fine tuning 데이터 로드
dataset = [
    {'messages': [
        {'role' : 'system', 'content' : """
        당신은 정부 지원 사업 전문 분석가입니다. 
        사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
        요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

        사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
        다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

        신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

        지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
        신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

        신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
        제공하는 템플릿 양식 : {base_template}
        단어 정의 사전 : {keyword_dictionary}
        참고 : {explanation}

        각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
        """},
        {'role' : 'user', 'content' : """
        '부천시공고제2025-191호2025년 소상공인 특례보증․이차보전 지원계획 공고  부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다. 2025년  1월 23일                                             부  천  시  장1. 사 업 명 : 2025년 소상공인 특례보증 지원    가. 출 연 금 : 12억 원    나. 보증규모 : 120억 원       ※상반기 60억원 / 하반기 60억원 분리운영    다. 보증대상 : 부천시 소재 소상공인      ▸사업자 등록일로부터 2개월 이상 경과 된 사업자    라. 보증한도 : 사업자별 최대 5,000만 원      ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수      ▸기존 보증금액 포함 5,000만원 이내      ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황),   재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정     마. 특례보증 제외대상      - 지방세, 세외수입 등 체납이 있는 사업자       - 제한업종(금융업 ․ 유흥업 ․ 불법도박 및 사치 ․ 향락업종 등)\n    바. 특례보증 신청 및 접수      - 접수기간         ․ 상반기 : 2025. 1. 23. ~ 자금소진시까지        ․ 하반기 : 2025. 7. 1. ~ 자금소진시까지      - 접 수 처 : 경기신용보증재단 부천지점        ․ 연락처 : 1577-5900        ․ 주  소 : 부천시 원미구 소향로 217, 4층      - 준비서류 : 사업자등록증 사본, 매출증빙서류, 신분증 등    사. 특례보증 처리 절차  상담․접수➡ 심사(서류․현장)➡보증서발급➡ 자금대출경기신용보증재단경기신용보증재단경기신용보증재단은행2. 사 업 명 : 2025년 소상공인 이차보전 지원    가. 대상 : 소상공인 특례보증 대상자    나. 내용 : 특례보증과 연계하여 대출이자 연 2% 1년간 지원      ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수    다. 협약은행 : 6개 관내 금융기관      ▸농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행 ○ 기타 자세한 문의는 경기신용보증재단 부천지점(☎ 1577-5900)    또는 부천시 지역경제과(☎ 032-625-2766)로 문의하시기 바랍니다.'
         """},
        {'role' : 'assistant', 'content' : """
        📌 [경기] 부천시 2025년 소상공인 특례보증ㆍ이차보전 지원계획 공고 요약 안내
        
        🤖 사업 개요 
        - 부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다.
    
        ✅ 사업자 유형
        - 개인사업자
        - 기존사업자

        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고 특성
        - 소상공인
        - 대출

        👥 지원대상
        1. 2025년 소상공인 특례보증 지원 : 부천시 소재 소상공인
         - 사업자 등록일로부터 2개월 이상 경과 된 사업자
        2. 2025년 소상공인 이차보전 지원 : 소상공인 특례보증 대상자

        📅 신청기간
        - 상반기: 2025. 1. 23. ~ 자금 소진 시까지
        - 하반기: 2025. 7. 1. ~ 자금 소진 시까지

        📝 신청방법
        - 특례보증 접수처 : 경기신용보증재단 부천지점
        - 주소 : 부천시 원미구 소향로 217, 4층

        🚫 지원제외 대상
        - 지방세, 세외수입 등 체납이 있는 사업자
        - 제한업종(금융업, 유흥업, 불법도박 및 사치, 향락업종 등)

        ⭐️ 우대사항

        👩 여성 대상
        X

        💁 청년 대상 
        X

        ⚠ 유의사항

        📑 제출서류
        - 사업자등록증 사본
        - 매출증빙서류
        - 신분증 등

        📌 선정절차
        - 상담․접수 ➡ 심사(서류․현장) ➡ 보증서발급 ➡ 자금대출

        💰 지원내용
        1. 2025년 소상공인 특례보증 지원
        - 보증한도 : 사업자별 최대 5,000만 원
        * 사업장 휴·폐업, 사업장 타 시·군 이전 시 지원 중지 및 환수
        * 기존 보증금액 포함 5,000만원 이내
        * 기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황), 재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정
        2. 2025년 소상공인 이차보전 지원
        - 특례보증과 연계하여 대출이자 연 2% 1년간 지원
        * 사업장 휴·폐업, 사업장 타 시·군 이전 시 지원 중지 및 환수
        - 협약은행 : 6개 관내 금융기관
        * 농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행

        📞 문의처
        - 경기신용보증재단 부천지점: 1577-5900
        - 부천시 지역경제과: 032-625-2766
        ```
    """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : """
        당신은 정부 지원 사업 전문 분석가입니다. 
        사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
        요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

        사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
        다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

        신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

        지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
        신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

        신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
        제공하는 템플릿 양식 : {base_template}
        단어 정의 사전 : {keyword_dictionary}
        참고 : {explanation}

        각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
        """},
        {'role' : 'user', 'content' : """
        "경제일자리재단 공고 제2025-3호「’25년 1인 자영업자 및 소상공인 사회보험료 지원사업」 공고 경기침체로 어려움을 겪는 소상공인의 경영난 해소와 근로자 고용안정망 구축을 위하여 ‘1인 자영업자 및 소상공인 사회보험료 지원사업’의 참여자를 다음과 같이 모집 공고합니다. 2025년 2월 17일(재)광주경제진흥상생일자리재단 대표이사1 사업개요 ❍ 사 업 명 : 1인 자영업자 및 소상공인 사회보험료 지원사업 ❍ 사업기간 : 2025. 1. ~ 사업비 소진 시까지 ❍ 지원대상   - (1인자영업자) 고용 및 산재보험법에 따라 기(신규) 가입한 1인 자영업자  - (소상공인) 정부 두루누리 사업장으로 관내 10인 미만 근로자를 고용한 사업장의 사업주 ❍ 지원내용  - (1인자영업자) 부담한 고용보험료 부과액의 20%, 산재보험료 부과액의 50%  - (소상공인) 근로자의 고용산재 보험료 중 정부지원을 제외한 사업주 부담분(고용보험 20%, 산재보험 100%) 지원 ❍ 수행기관 : (재)광주광역시 경제진흥상생일자리재단\n2 세부내용 \uf0001인 자영업자 사회보험료 지원사업 ❍ 사업기간 : ’25. 1. 1. ~ 사업비 소진 시까지 ❍ 지급절차 : 1인 자영업자가 월별 보험료 선납 -> 월별 납부내역 확인 후 지원금 지급 ❍ 지급시기 : 매월말까지 확정된 지원대상자에 대하여 월별 지급고용・산재보험 가입 및 납부➡온·오프라인 신청(기업지원시스템·이메일 접수, 방문접수)➡고용·산재보험  납부내역 확인 및 지급대상 검토➡사실관계 확인 후 보험료 지원금 지급1인자영업자1인자영업자→경제진흥상생일자리재단경 제 진 흥 상 생 일 자 리 재 단→근로복지공단경제진흥상생일자리재단→1인자영업자기납부연중연중월별 지급  * 예) 1월 사회보험료 지원금은 1월 보험료의 법정 납부기한(2월 10일경)의 익월인 3월 지급 ❍ 지급대상 : 고용 및 산재보험법에 따라 기(신규) 가입한 1인 자영업자 (고용보험 기준보수 1~7등급 / 산재보험 기준보수 1~12등급까지 전 등급 지원)  * 광주소재사업장에 한하며, 근로복지공단의 보험 가입대상을 기본조건으로 함     공동사업자의 경우 대표 1인만 지원 ❍ 지급방법 : 사업주 명의 계좌로 지원금 지급 ❍ 신청방법 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청  * 온라인 신청 : https://gjbizinfo.or.kr(광주기업지원시스템)  * 최초 1회 신청, 사업자 변경(사업주, 사업자등록번호 등)되는 경우 변경신청 필수  * 기신청했던 업체에서는 신청자 변동 없을 시 별도 신청 없이 지급 예정  * 오프라인 신청 : 방문접수(경제진흥상생일자리재단 3층 소상공인디지털전환실)  ❍ 제출서류 : 자영업자 고용ž산재보험에 가입 후 아래의 서류 제출구 분 제출서류비고필수제출서류①\x00사업신청서(행정정보공동이용동의서포함)[서식1]②중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]\x00③통장사본(스마트폰캡처본도인정) 수수료없음선택제출서류①사업자등록증또는사업자등록증명원사본1부.②\x00건강보험자격득실확인서사본1부.(신청일로부터1개월이내발급본)\x00*\x00국민건강보험공단홈페이지(www.nhis.go.kr)\x00또는정부24에서발급가능행정정보공동이용미동의시별도제출\n  * 추가 서류제출이 필요한 경우v 사업자등록증의 상호와 건강보험자격득실확인서상 사업장명칭이 동일할 경우 :‘사 업 장  가 입 자 별  부 과 현 황’  또 는  ‘개 인 별  건 강 보 험  고 지 산 출 내 역’ (최 근  2개 월  이 내 의  고 지 연 월 로  발 급) 제 출 \uf000소상공인 사회보험료 지원사업 ❍ 사업기간 : ’25. 1. 1. ~ 사업비 소진 시까지 ❍ 지급절차 : 사업자가 월별 보험료 선납 -> 월별 납부내역 확인 후               지원금 지급두루누리사회보험지원신청➡보험료先납부➡서류접수(온ž오프라인)➡고용·산재보험 납부내역 확인 및 지급대상 검토➡보험료 지원금지급사업주 ->근로복지공단사업주사업주 ->재단재단 <-> 근로복지공단재단 -> 사업주 ❍ 지급시기 : 매월말까지 확정된 지원대상자에 대하여 월별 지급  * 단, 1월 지원금은 1월 보험료의 법정 납부기한(2월 10일경)의 다음달인 3월부터 지급  * 두루누리 사회보험료 지원을 받은 해당연도에 한하여 市 사회보험료를 지원    하고 해당 월 보험료를 법정 납부기한까지 완납한 건만 지원 ❍ 지급대상 : 정부 두루누리 사업장으로 관내 10인 미만 근로자를 고용한 사업장의 사업주  * 지원제외 : 사업주 본인, 건설·벌목업, 예술인, 특수형태 근로종사자 사업장,               사업주의 배우자 및 직계존비속 근로자의 보험료 미지원 ❍ 지급조건  ① 근로자 수가 10명 미만인 사업장(사업주) * 두루누리 사회보험료 지원여부로 확인   ‧ 지원신청일이 속한 보험연도의 전년도에 근로자인 피보험자 수가 월평균 10명    미만이고 지원신청 월 말 기준 10명 미만인 사업장   ‧ 지원신청일이 속한 보험연도의 전년도에 근로자인 피보험자 수가 월평균 10명    이상이나 지원신청 월 직전 3개월 동안 연속 10명 미만인 사업장   ‧ 지원신청일이 속한 보험연도 중에 보험관계가 성립한 사업으로 지원신청 월 직전    3개월 동안 근로자인 피보험자 수가 연속 10명 미만인 사업장\n  ② 월평균 보수액 270만원 미만 근로자 고용 * 두루누리 사회보험료 지원여부로 확인   ‧ 「소득세법」에 따른 근로소득에서 비과세 근로소득을 공제한 총급여액을    기준으로 월 평균액이 270만원 미만   ‧ 신규가입자 근로복지공단 납부내역과 4대보험 사업장 가입자명부로 확인 ❍ 신청방법 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청  * 온라인 신청 : https://gjbizinfo.or.kr(광주기업지원시스템), 담당자 이메일 접수  * 기존에 신청했던 업체에서는 신청자 변동 없을 시 별도 신청없이 지급 예정  * 오프라인 신청 : 방문접수(경제진흥상생일자리재단 3층 소상공인디지털전환실) ❍ 제출서류구 분신청주기제출서류신규신청최초1회①소상공인고용·산재보험료지원신청서[서식2]②사회보험료지원체크리스트[서식3]③중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]\x00④개인정보수집·활용동의서(근로자용)\x00[첨부]\x00/\x00보험료지원신청근로자전체⑤사업자등록증,\x00사업자등록증명원,\x00고유번호증(비영리단체)\x00사본⑥\x00사업주또는사업장명의통장사본(스마트폰캡처본도인정)⑦4대보험사업장가입자명부변경신청근로자추가시(추가신청자만해당)①소상공인고용·산재보험료지원신청서[서식2]②사회보험료지원체크리스트[서식3]③개인정보수집·활용동의서(근로자용)\x00[첨부]\x00\x00/\x00변경신청근로자전체【참 고】○ 고용보험이란?   - 「고용산재보험료 징수법」 제49조의2(자영업자에 대한 특례)에 따라, 사업주가       고용보험에 가입할 수 있는 제도   - (가입혜택) 비자발적 사유로 폐업 시 실업급여 지급, 직업능력개발훈련비 지원○ 산재보험이란?   - 「산업재해보상보호법」 제124조(중소기업 사업주에 대한 특례)에 따라,      사업주가 산재보험에 가입할 수 있는 제도   - (가입혜택) 업무상 사유로 부상, 질병, 사망 등의 재해를 입은 경우 산재보험급여 지급\n3 문의처 ❍ 1인 자영업자 사회보험료 지원사업구분담당기관전화번호고용·산재보험 광주광역시 지원문의(재)경제진흥상생일자리재단 소상공인디지털전환실062-960-2632고용·산재보험 신규가입 문의근로복지공단1588-0075고용보험 정부지원 문의소상공인시장진흥공단1357 ❍ 소상공인 사회보험료 지원사업구분담당기관전화번호사회보험료광주광역시 지원문의(재)경제진흥상생일자리재단 소상공인디지털전환실062-960-2632두루누리 사회보험료지원문의근로복지공단1588-0075국민연금공단1355\n서식 1     '25년 1인 자영업자 사회보험료(고용,산재보험) 지원신청서(앞쪽)접수번호접수일자\n신청인대표자명주민번호휴대전화업체명사업장주소사업자등록번호업종가입여부고용□ 가입  □ 미가입본인제외 상시근로자 유무□ 있음  □ 없음산재□ 가입  □ 미가입지원금지급계좌(사업주명의) 은행명예금주(신청인) 계좌번호 * 지원금 지급 계좌는 첨부 서류의 통장 사본과 반드시 일치하여야 함「소상공인 보호 및 지원에 관한 법률」 제12조의7 및 광주광역시 소상공인지원조례 제6조에 따라 위와 같이 신청합니다.  2025년      월      일신청인(서명 또는 인)  (재)광주경제진흥상생일자리재단 대표이사  귀하필수제출서류1.\x00사업신청서(행정정보공동이용동의서포함)2.\x00통장사본(스마트폰캡처본도인정)3.\x00중소기업통합관리시스템기업정보수집·이용·제공동의서수수료없음선택제출서류1.\x00사업자등록증또는사업자등록증명원사본1부.2.\x00건강보험자격득실확인서사본1부.(신청일로부터1개월이내발급본)\x00\x00*\x00국민건강보험공단홈페이지(www.nhis.go.kr)\x00또는정부24에서발급가능행정정보공동이용미동의시별도제출행정정보공동이용동의서 본인은 이 건 업무처리와 관련하여 「전자정부법」 제36조제1항에 따른 행정정보의 공동이용을 통하여 담당공무원이 위의 사항을 확인하는 것에 동의합니다.                                              신청인                        (서명 또는 인)\n서식2 '25년 사회보험료(고용, 산재보험) 지원 신청서 (□ 신규 □ 변경)※ 뒷면의 작성방법 및 유의사항을 반드시 확인하여 주시기 바랍니다. (앞면) 접수번호 접수일신청인업체명사업자등록번호사업장관리번호(*필수)(고용보험)(산재보험)사업장주소 대표자명/주민번호담당자명연락처(사무실)이메일주소 연락처(휴대폰)사회보험료 지원 신청 근로자연번성  명생년월일(8자리) 성별퇴사자 체크1234필수제출서류1.\x00소상공인고용·산재보험료지원신청서[서식2]\x002.\x00사회보험료지원부정수급방지체크리스트[서식3]\x003.\x00중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]\x004.\x00개인정보수집·활용동의서(근로자용)\x00[첨부]\x00/\x00보험료지원신청근로자전체5.\x00사업자등록증,\x00사업자등록증명원,\x00고유번호증(비영리단체)\x00사본6.\x00사업주또는사업장명의통장사본(스마트폰캡처본도인정)7.\x004대보험사업장가입자명부※서류는최초신청1회만제출,\x00단,사업장변경(사업주,\x00사업자등록번호,\x00근로자등)되는경우변경신청필수선택제출서류사업자등록증행정정보공동이용미동의시별도제출(필수) 지원대상자 중 사업주(대표자), 사업주(대표자)의 배우자, 직계존비속이 없음을 확인하며, 신청내용에 허위나 거짓이 있을 경우나 부적격자에게 지원금이 지급되거나 잘못 지급된 경우, 지원대상자가 정부 및 지자체 등의 다른 보험료 지원과 중복 수령 시 지원금 전액을 환수하고, 민·형사상 책임을 지는 것에 동의합니다.       ※ 동의하지 않는 경우 지원금이 지급되지 않습니다.                   (동 의  □ 부 동 의  □)(필수) 타지역으로 사업장을 이전한 경우, 지원금이 자동 종료됩니다.                             (확 인  □ )                                                                                     2025년       월       일신청인(사업주)                   (서명 또는 인)공동신청인(공동사업주)                   (서명 또는 인)(재)광주광역시 경제진흥상생일자리재단 대표이사 귀하행정정보 공동이용 동의서   본인은 이 건 업무처리와 관련하여 「전자정부법」 제36조제1항에 따른 행정정보의 공동이용을 통하여 담당공무원이 위의 사항을 확인하는 것에 동의합니다.                                                     신청인                        (서명 또는 인)\n작  성  방  법1.최초 지원 신청하는 사업장은 신규 란에 표기하여 주시고, 기존에 신청했던 사업장 중 변경사항이 있는 사업장은 변경란에 표기하여야 합니다.※ 기존 신청 사업장 중 변경이 없는 사업장은 표기하지 않습니다.2. 사회보험별 사업장 관리번호는 사업장에 따라 모두 동일한 경우도 있고, 각각 부여되어 있는 경우도     있으므로 신청인이 사회보험료 별로 확인 후 정확하게 기재해야 합니다. ※근로복지공단(1588-0075) 3. 지원 신청 대상자는 사업장 소속 근로자 중 월평균 보수액 270만원 미만으로 정부의 두루누리     사회보험 지원을 받고 있으며, 1개월 이상 고용을 유지하고 있는 근로자와 퇴사자(퇴사 전    1개월 이상 고용유지, 두루누리 사회보험 지원 필수)를 포함하여 기재해야 합니다.4. 공동사업주가 있을 경우 공동사업주 현황을 기재하여 주시고, 개인정보 수집‧이용 및 제3자    제공 동의서를 별도로 제출해야 합니다.유  의  사  항1.두루누리 사회보험 지원금을 받은 월에 한하여 광주광역시 사회보험료를 지원하며, 두루누리 사회보험 지원금은 해당 월 보험료를 법정 납부기한까지 완납하여야만 지원됩니다.※ 법정 납부기한 후에 보험료를 완납하여도 두루누리 사회보험 지원금은 지원되지 않습니다.2. 사회보험료가 체납된 사업장은 사회보험료 지원을 받을 수 없습니다.3. 신청 사업장의 신규 입사자가 사회보험료 지원 최초 신청 시 개인정보 수집‧이용 및 제3자 제공   동의서를 제출해야 하며, 퇴사자에 대한 지원 신청 시 퇴사자도 개인정보 수집‧이용 및 제3자    제공동의서를 제출해야 지원이 가능합니다.처  리  절  차신청서 등 작성및 제출è접수 및 확인è신청자별 보험료부과내역 확인è지원금 지급여부안내 및 지급è지원금 수령[신청인][경 제 진 흥 상 생일 자 리 재 단][광 주 광 역 시  및  근 로 복 지 공 단][경 제 진 흥 상 생일 자 리 재 단][신청인]\n서식3 사회보험료(고용, 산재보험) 지원 부정수급방지 체크리스트q 광주광역시 사회보험료 지원사업은 최저임금 인상에 따른 사업주의 경영부담 완화를 위해  사회보험료 사업주 부담분을 지원하는 사업입니다.q 본 지원금은 아래 사업장과 근로자 요건을 모두 충족하여야 지급됨을 알려 드리며, 요건 충족 시 굵은 선 안의 체크표시에 진하게 표시해 주시면 됩니다.   (1개 문항이라도 “아니요”에 해당할 경우 지원 사업 참여 불가)q 사업장 요건 예아니오1. 사회보험료 지원 신청을 한 월부터 매월 말일 기준 근로자수가 10인 미만입니다.2. 10인 미만 요건 충족을 위해 인위적 감원이나 사업장을 분할하지 않았습니다.3. 임금체불 중 또는 임금체불로 명단 공개중이 아닙니다.4. 국가 및 공공기관에 해당하지 않습니다. 5. 두루누리 지원을 제외한 다른 사회보험료 지원을 받고 있지 않습니다. 6. 국가나 지자체로부터 최저임금 인상에 따라 추진되는 다른 지원 사업으로   재정지원을 받고 있지 않습니다. (일자리 안정자금 지원 제외)7. 근로자의 신규 입사 및 퇴사의 경우 각 사회보험별 자격 취득 및 상실   신고와 보수 변경 시 보수월액변경신고 등을 준수하고 있습니다. q 근로자 요건(반드시 개인별로 판단하여야 합니다.) 예아니오1.지원 신청대상 근로자는 월평균 보수액이 270만원 미만이며   정부의 두루누리 지원을 받고 있습니다.2. 지원 신청대상 근로자는 모두 1개월 이상 고용을 유지하고 있습니다.3. 지원 신청대상 근로자는 사업주, 사업주의 배우자 또는 사업주의   직계존비속이 아닙니다.4. 사회보험 가입대상인 근로자는 사회보험에 가입되어 있습니다. 5. 기존 근로자는 최소한 전년도 보수수준을 유지하고 있습니다.□ 유의사항 m 지원기간 동안 새로운 근로계약으로 인해 월평균 보수액이 270만원 이상이 된 근로자는 지원신청 하실    수 없습니다.   위 내용을 충분히 이해하였으며, 거짓이나 그 밖의 부정한 방법으로 광주광역시 사회보험료 지원 신청을 하여 지원금을 수령할 시 지원금 환수와 더불어 필요 시 민·형사상 책임을 질 수 있음에 동의합니다.    2025년    월    일신청인(사업주):            (서명 또는 인) 공동신청인(공동사업주):            (서명 또는 인)(재)광주광역시 경제진흥상생일자리재단 대표이사 귀하\n첨  부 사회보험료(고용, 산재보험) 지원신청을 위한개인정보 수집·이용 및 제3자 제공 동의서(근로자용)소상공인을 대상으로 고용・산재보험료의 일부를 지원하고 광주광역시 소상공인 지원사업 문자알림 서비스제공을 위하여 귀하의 개인정보를 수집·이용하고자 하며, 아울러 『고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률』에 따라 고용・산재보험에 가입한 근로복지공단으로부터 개인정보를 제공받고자 『개인정보 보호법』 제15조제1항제1호, 제17조제1항제1호, 제18조제2항 제1호, 제3항에 따라 귀하의 동의를 받고자 하오니, 아래의 내용을 자세히 읽어보신 후 동의 여부를 결정하여 주시기 바라며 동의를 거부할 경우 사회보험료 지원에 제한을 받을 수 있습니다.□ 개인정보 수집․이용 내역항     목 수 집 목 적보 유 기 간성명, 생년월일, 성별,  월 보수액 및 보수총액, 개인별 사회보험료 산출·부과·고지·납부 내역 및 사회보험료 정부지원내역, 소속 사업장 관리번호 등 광주광역시 사회보험료 신청(지원) 사업장의 지원 적합여부 및 지원 금액 확인해당 사업장의 광주광역시 사회보험료 지원 종료 시까지광주광역시 소상공인 지원사업 정보 제공2년□ 위와 같이 개인 및 사업장 정보를 수집·이용하는데 동의하십니까? \uf000동의  \uf000미동의□ 개인 및 사업장 정보 제3자 제공내역제공하는 자제공받는 자제  공  항  목제  공  목  적보 유 기 간광주광역시,광주경제진흥상생일자리재단(수행기관)근로복지공단성명, 생년월일,사업장 관리번호신청 사업장이 부담하고 있는 근로자의 월 보수액, 사업장 부담 사회보험료 산출·부과·납부내역 및 사회보험료 지원 내역, 사회보험 취득·상실일 확인해당 사업장의 광주광역시 사회보험료 지원 종료 시까지성명, 연락처광주광역시 소상공인 지원사업 정보 문자알림2년□ 위와 같이 개인 정보를 수집·이용하고 제3자에게 제공하는데 동의하십니까?\uf000동의  \uf000미동의□ 개인 및 사업장 정보 제3자 제공내역제공하는 자제공받는 자제  공  항  목제  공  목  적보 유 기 간근로복지공단광주광역시,광주경제진흥상생일자리재단(수행기관)광주광역시가 제공한 사업장의 근로자의 개인별 사회보험료 산출․부과·납부내역과 사회보험료 지원내역광주광역시 사회보험료 신청(지원) 사업장의 사회보험료 사업주 부담금 확인 및 사회보험료 지원금 산출에 활용해당 기업에 대한 광주광역시의사회보험료 지원 사업 지원 종료 시 까지□ 위와 같이 개인정보를 수집·이용 하고 제3자에게 제공하는데 모두 동의하십니까?                                                                         \uf000동의  \uf000미동의2025년   월   일사업장명                근로자 성명        (서명 또는 인)광주광역시 ․ 광주경제진흥상생일자리재단·근로복지공단 귀중\n서식4 중소기업 지원사업 통합관리시스템기업(신용)정보 수집·이용·제공 동의서❶ 수집·이용에 관한 사항수집·이용목적\uf000 중소기업시책에 참여하는 기업의 지원효과 분석 \uf000 중소기업 지원사업의 효율화(만족도 조사)수집·이용항목\uf000 중소기업기본법 제20조의2에 따른 기업 소재지, 업종, 매출액, 납입자본금, 자산총액, 부채총액, 영업이익, 당기순이익, 개업일·휴업일·폐업일, 전자계산서 발급액, 전자지급 거래액, 일반연구ˑ인력개발비, 현금영수증 결제금액, 신용카드 결제금액, 신고한 수출 물품의 품명, 품목 번호, 총 신고가격, 목적지, 신고일\uf000 중소기업 지원사업에 대한 만족도 조사 목적의 지원이력 정보(전화번호, 이메일 등)수집·이용기관 · 중소벤처기업부수집·이용기간 · 중소기업 지원사업 참여 기준 이전 3개년부터 사업 참여 이후 5년까지❷ 기업(신용)정보의 파기파기대상 정보 · 국세청 및 관세청 과세정보 · 개인을 식별할 수 있는 전화번호, 이메일 등파기절차 및 방법 · 파기 계획을 수립하여, 수집 및 이용기간 경과 시 30일 이내 지체 없이 파기 · 복원이 불가능한 방법으로 영구 삭제 본인은 위 목적으로 동의서에 적시된 정보 및 기관에 한해 기업정보를 수집‧이용‧제공하는 것에 동의합니다.※본동의서징구는민법제114조의행정행위의대리권행사방식의현명주의를적용하여,중소기업지원사업의효율적운영을위해본사업의소관부처및수행기관이대신하여받는것임을알려드립니다.※귀하는기업정보제공에대한동의를거부할권리가있습니다.동의를거부할경우지원사업맞춤형서비스이용에제한을받을수있습니다. 2025년월일 기업명ㅇㅇㅇ 대표자ㅇㅇㅇ(인)"
        """},
        {'role' : 'assistant', 'content' : """
        📌 [광주] 2025년 1인 자영업자 및 소상공인 사회보험료 지원사업 요약 안내
         
        🤖 사업 개요
        - 경기침체로 어려움을 겪는 소상공인의 경영난 해소와 근로자 고용안정망 구축을 위하여 ‘1인 자영업자 및 소상공인 사회보험료 지원사업’의 참여자를 다음과 같이 모집 공고합니다.

        ✅ 사업자 유형
        - 개인사업자
        - 기존사업자
         
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 소상공인
        - 두루누리지원금
         
        👥 지원대상
        - (1인 자영업자) 고용 및 산재보험법에 따라 기(신규) 가입한 1인 자영업자
        - (소상공인) 정부 두루누리 사업장으로 관내 10인 미만 근로자를 고용한 사업장의 사업주

        📅 신청기간
        - 25. 1. 1. ~ 사업비 소진 시까지

        📝 신청방법
        - 1) 1인 자영업자 사회보험료 지원사업 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청
        - 온라인 신청 : gjbizinfo.or.kr (광주기업지원시스템)
        * 최초 1회 신청, 사업자 변경(사업주, 사업자등록번호 등)되는 경우 변경신청 필수
        * 기신청했던 업체에서는 신청자 변동 없을 시 별도 신청 없이 지급 예정
        - 오프라인 신청 : 방문 접수 (경제진흥상생일자리재단 3층 소상공인디지털전환실) 
        2) 소상공인 사회보험료 지원사업 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청
        - 온라인 신청 : gjbizinfo.or.kr (광주기업지원시스템), 담당자 이메일 접수
        * 기존에 신청했던 업체에서는 신청자 변동 없을 시 별도 신청없이 지급 예정
        - 오프라인 신청 : 방문 접수 (경제진흥상생일자리재단 3층 소상공인디지털전환실)
         
        🚫 지원제외 대상
        - (소상공인) 업주 본인, 건설·벌목업, 예술인, 특수형태 근로종사자 사업장, 사업주의 배우자 및 직계존비속 근로자의 보험료 미지원
         
        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X
         
        ⚠ 유의사항
        1. 두루누리 사회보험 지원금을 받은 월에 한하여 광주광역시 사회보험료를 지원하며, 두루누리 
        사회보험 지원금은 해당 월 보험료를 법정 납부기한까지 완납하여야만 지원됩니다.
        ※ 법정 납부기한 후에 보험료를 완납하여도 두루누리 사회보험 지원금은 지원되지 않습니다.
        2. 사회보험료가 체납된 사업장은 사회보험료 지원을 받을 수 없습니다.
        3. 신청 사업장의 신규 입사자가 사회보험료 지원 최초 신청 시 개인정보 수집‧이용 및 제3자 제공
        동의서를 제출해야 하며, 퇴사자에 대한 지원 신청 시 퇴사자도 개인정보 수집‧이용 및 제3자 
        제공동의서를 제출해야 지원이 가능합니다.
         
        📑 제출서류    
        1인 자영업자 사회보험료 지원사업 제출서류
        | 구분          | 제출서류                                                                 | 비고                                                                 |
        |---------------|--------------------------------------------------------------------------|----------------------------------------------------------------------|
        | 필수 제출서류 | ① 사업신청서(행정정보공동이용동의서 포함) [서식1]                      |                                                                      |
        |               | ② 중소기업통합관리시스템 기업정보수집·이용·제공동의서 [서식4]          |                                                                      |
        |               | ③ 통장사본(스마트폰 캡처본도 인정)                                      | 수수료 없음                                                          |
        | 선택 제출서류 | ① 사업자등록증 또는 사업자등록증명원 사본 1부                          |                                                                      |
        |               | ② 건강보험자격득실확인서 사본 1부 (신청일로부터 1개월 이내 발급본)     | * 국민건강보험공단 홈페이지(www.nhis.go.kr) 또는 정부24에서 발급 가능 |
        |               |                                                                          | 행정정보공동이용 미동의 시 별도 제출               

         소상공인 사회보험료 지원사업 제출서류

        | 구분          | 신청주기 | 제출서류                                                                 | 비고 |
        |---------------|----------|--------------------------------------------------------------------------|------|
        | 신규 신청    | 최초 1회 | ① 소상공인 고용·산재보험료 지원신청서 [서식2]                           |      |
        |               |          | ② 사회보험료 지원 체크리스트 [서식3]                                    |      |
        |               |          | ③ 중소기업통합관리시스템 기업정보수집·이용·제공동의서 [서식4]          |      |
        |               |          | ④ 개인정보수집·활용동의서(근로자용) [첨부] / 보험료 지원신청 근로자 전체 |      |
        |               |          | ⑤ 사업자등록증, 사업자등록증명원, 고유번호증(비영리단체) 사본          |      |
        |               |          | ⑥ 사업주 또는 사업장 명의 통장사본(스마트폰 캡처본도 인정)             |      |
        |               |          | ⑦ 4대보험 사업장 가입자 명부                                           |      |
        | 변경 신청    | 근로자 추가 시 (추가 신청자만 해당) | ① 소상공인 고용·산재보험료 지원신청서 [서식2]                           |      |
        |               |          | ② 사회보험료 지원 체크리스트 [서식3]                                    |      |
        |               |          | ③ 개인정보수집·활용동의서(근로자용) [첨부] / 변경신청 근로자 전체      |      |
         
        📌 선정절차
        - (1인 자영업자) 고용・산재보험 가입 및 납부 ➡ 온·오프라인 신청(기업지원시스템·이메일 접수, 방문접수) ➡ 고용·산재보험 납부내역 확인 및 지급대상 검토 ➡ 사실관계 확인 후 보험료 지원금 지급
        - (소상공인) 두루누리 사회보험 지원신청 ➡ 보험료 先납부 서류접수(온·오프라인) ➡ 고용·산재보험 납부내역 확인 및 지급대상 검토 ➡ 보험료 지원금 지급
         
        💰 지원내용
        - (1인 자영업자) 부담한 고용보험료 부과액의 20%, 산재보험료 부과액의 50%
        - (소상공인) 근로자의 고용산재 보험료 중 정부지원을 제외한 사업주 부담분(고용보험 20%, 산재보험 100%) 지원
         
        📞 문의처
        - 1인 자영업자 사회보험료 지원사업 : 
        | 구분                       | 담당기관                     | 전화번호   |
        |:---------------------------|:-----------------------------|:-----------|
        | 고용·산재보험 광주광역시 지원문의 | (재)경제진흥상생일자리재단 소상공인디지털전환실 | 062-960-2632 |
        | 고용·산재보험 신규가입 문의       | 근로복지공단                   | 1588-0075  |
        | 고용보험 정부지원 문의           | 소상공인시장진흥공단             | 1357       |
        - 소상공인 사회보험료 지원사업 :
        | 구분                       | 담당기관                     | 전화번호   |
        | 사회보험료 광주광역시 지원문의     | (재)경제진흥상생일자리재단 소상공인디지털전환실 | 062-960-2632 |
        | 두루누리 사회보험료 지원문의       | 근로복지공단                   | 1588-0075  |
        | 국민연금공단                 | 근로복지공단                   | 1355       |
        """},
        ]},
    {'messages': [
        {'role' : 'system', 'content' : """
        당신은 정부 지원 사업 전문 분석가입니다. 
        사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
        요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

        사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
        다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

        신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

        지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
        신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

        신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
        제공하는 템플릿 양식 : {base_template}
        단어 정의 사전 : {keyword_dictionary}
        참고 : {explanation}

        각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
        """},
        {'role' : 'user', 'content' : """
        '- 16 -\n□ 창업기반지원자금(청년전용창업자금) ◦ (대상) 대표자가만39세이하로서, 업력3년미만인중소기업 또는중소기업을창업하는자     * 단, 창업성공패키지사업(청년창업사관학교, 글로벌창업사관학교) 참여 기업, 청년창업기업보증 지원 기업(기보) 및 민간VC 투자 유치 기업 중 해당 기관의 추천을 받은 우수 기업은 업력 7년 미만까지 가능    ** 청년전용창업자금 대상 중 중점지원분야(참고1~4)를 영위하는 소상공인은 청년전용창업자금 신청 가능  ◦ (융자방식) 직접대출  ◦ (대출조건)구분대출 조건대출한도기업당 최대 1억원 이내 (제조업 및 중점지원분야 영위기업은 2억원 이내)대출기간(직접대출, 시설) 10년 이내 (거치기간 : 담보 4년 이내, 신용 3년 이내)(직접대출, 운전)  6년 이내 (거치기간 : 3년 이내)대출금리2.5% (고정금리) 기    타자금신청‧접수 후, 사업계획서 등 심의위원회 평가를 통해지원 여부 결정 후 대출※ 성장공유형 대출 및 투자조건부 융자의 세부 지원대상, 지원조건, 지원절차 등은 추후 공고'
        """},
        {'role' : 'assistant', 'content' : """
        📌 혁신창업사업화자금-창업기반지원자금(청년전용창업자금)(2025년 중소기업 정책자금 융자계획 공고) 요약 안내
            
        🤖 사업 개요
        - 

        ✅ 사업자 유형
        - 법인사업자
        - 기존사업자
            
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 소상공인
        - 청년대상
        - 대출
        - 창업패키지
            
        👥 지원대상
        ㅇ 신청대상 : 대표자가 만 39세 이하로서 , 업력 3년 미만인 중소기업 또는 중소기업을 창업하는 자
        * 단, 창업성공패키지사업(청년창업사관학교, 글로벌창업사관학교) 참여 기업, 청년창업기업보증 지원 기업(기보) 및 민간VC 투자 유치 기업 중 해당 기관의 추천을 받은 우수 기업은 업력 7년 미만까지 가능
        ** 청년전용창업자금 대상 중 중점지원분야를 영위하는 소상공인은 청년전용창업자금 신청 가능

        📅 신청기간
        자금 소진시까지

        📝 신청방법
        - 
            
        🚫 지원제외 대상
        -
            
        ⭐️ 우대사항
        - 
            
        👩 여성 대상
        X
            
        💁 청년 대상
        O
            
        ⚠ 유의사항
        - 
            
        📌 선정절차
        - 
            
        💰 지원내용
        | 구분 | 대출 조건 | 대출 한도 |
        |------|-----------|------------|
        | 직접대출, 시설 | 10년 이내 (거치기간 : 담보 4년 이내, 신용 3년 이내) | 기업당 최대 1억원 이내 (제조업 및 중점지원분야 영위기업은 2억원 이내) |
        | 직접대출, 운전 | 6년 이내 (거치기간 : 3년 이내) | 기업당 최대 1억원 이내 (제조업 및 중점지원분야 영위기업은 2억원 이내) |
            
        📞 문의처
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : """
        당신은 정부 지원 사업 전문 분석가입니다. 
        사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
        요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

        사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
        다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

        신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

        지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
        신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

        신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
        제공하는 템플릿 양식 : {base_template}
        단어 정의 사전 : {keyword_dictionary}
        참고 : {explanation}

        각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
        """},
        {'role' : 'user', 'content' : """
        '\x02\x00\x00\x00\x00\x02\x02\x00\x00\x00\x00\x02\x15\x00\x00\x00\x00\x15전라남도 공고 제2025 - 8호\r\n2025년도 전라남도 중소기업 육성자금 지원계획 공고\r\n  전라남도 중소기업 육성기금 조례제12조의 규정에 따라 2025년도 전라남도 중소기업 육성자금 지원계획을 다음과 같이 공고합니다.\r\n2025. 1.  2.\r\n전라남도지사\r\n\x0b\x00\x00\x00\x00\x0b\r\n5\r\n \r\n중소유통업 구조개선자금 지원 \r\n 1. 지원규모 : 30억원\r\n    은행상담 후 자금 신청을 하여야 하며, 은행 여신규정에 의한 담보(보증서 등) 제공이 있어야 대출을 받을 수 있음.\r\n 2. 지원대상\r\n   지원대상 업종\r\n    - 지원 신청일 현재 도내에 사업자등록을 한 업체\r\n      한국표준산업분류 중 도소매업 및 상품중개업자\r\n      프랜차이즈 본부 및 가맹점(유통업에 한함)\r\n    - 전통시장, 중소백화점쇼핑센터 동의 개설자 또는 관리자\r\n지원 제외 대상\r\n   국세지방세 체납중인 기업    \r\n   휴폐업중인 업체\r\n   전라남도 중소기업 육성자금(시설자금) 이용 중이거나, 당해연도 추천 후 대출하지 않은 기업(단, 이용한도가 남은 기업은 한도 내 이용 가능)\r\n   당해 연도 정부 및 공공기관으로부터 시설자금 수혜기업\r\n   중소벤처기업부 중소기업지원사업 통합관리시스템에서 접수일 기준  최근 5년간 정부기관 및 지자체 등의 지원금액(사업분야 : 금융)과 본 신청 건을 포함하여 지원금액이 100억원을 초과하는 기업\r\n       금융  보증/보험 제외\r\n   기타 자금지원 관련 규정에서 제외하도록 정한 업체\r\n연속지원 제한\r\n   시설자금 상환 후 2년 이내, 운영자금 상환 후 1년 이내 융자 지원 제한\r\n 3. 지원내용\r\n\x0b\x00\x00\x00\x00\x0b\r\n구 분\r\n지원사업\r\n융자금액\r\n융자기간\r\n융자이율\r\n시설\r\n투자\r\n  전문상가 건립\r\n  전문상가 시설개선\r\n  공동창고 건립\r\n4억원 이내\r\n3년거치 \r\n5년 분할상환\r\n연3.0%\r\n(25.1분기 기준)\r\n- 정부정책자금 기준금리에 따라 분기별 변동 금리 적용\r\n  점포시설개선\r\n1억원 이내\r\n운영\r\n자금\r\n  경쟁력강화 및\r\n    경영활성화 자금\r\n1억원 이내\r\n1년거치 \r\n2년 분할상환\r\n 4. 지원신청 : 반기별 접수(공고일 ~, 6. 1. ~)\r\n 5. 접수방법 : 온라인 접수\r\n   - 전남중소기업일자리경제진흥원 자금시스템(\x03\x00\x00\x00\x00\x03http://www.jnfund.kr\x04h\x00\x00\x00\x00\x04) \r\n   - 문 의 처 : 061-288-3832~4 (기업육성팀)  \r\n 6. 구비서류\r\n   공통서류\r\n    가. 자금신청서(별지 제1호), 사업계획서(별지 제2호), 정보이용동의서(별지 제4호) 각 1부(원본)\r\n             법인인 경우 반드시 법인도장 날인, 개인사업자 중 공동대표의 경우 대표자 전원 도장 날인\r\n    나. 대출상담확인서(별지 제3호) 1부(원본)\r\n    다. 사업자등록증 1부(사본)\r\n    라. 사업장 확인서류 1부(사본)\r\n      - 자가 : 건축물대장(소유자 확인 필요)\r\n      - 임대 : 임대계약서 + 건축물대장(소유자 확인 필요)\r\n           법인인 경우, 사업장 소유자가 대표자일 때 법인과 대표자간 임대차계약서\r\n    마. 세무사 또는 공인회계사가 확인한 최근 1개년 재무제표(또는 표준재무제표증명원-국세청 홈택스 발행분) 1부 \r\n            결산이 완료되기 전인 1~3월경까지는 법인도 전년도분에 한해 부가가치세과세표준증명원으로 대체 가능  \r\n   사업별 추가 구비서류\r\n    가. 전문상가 시설개선사업\r\n      - 전문상가 관리 법인 또는 입점상인 단체를 증명하는 서류 1부\r\n      - 입점상인의 시설 개보수 동의서\r\n      - 개보수 시공업체 사업자등록증, 계약서, 견적서 1부\r\n    나. 전문상가 및 공동창고 건립사업\r\n      - 전문상가 관리 법인 또는 입점상인 단체를 증명하는 서류 1부\r\n      - 토지등기부등본, 건축허가서 1부\r\n      - 시공업체 사업자등록증 사본, 건축시공 계약서 사본, 설계내역서, 건축도면 각 1부\r\n    다. 점포시설개선사업 : 개보수시공업체 사업자등록증, 계약서, 견적서 각 1부\r\n    라. 운영자금 : 매입세금계산서/합계표(신청금액 상당액, 회계사 또는 세무사 확인 표지 포함), 매입처 사업자등록증(신청금액 상당액 해당업체) 사본 1부\r\n 7. 지원업체 선정\r\n  전라남도 중소기업 육성기금 조례 시행규칙제13조의 규정에   따라 제외 대상업체 및 결격 여부를 심사하여 융자대상 업체 선정\r\n 8. 대출기한 : 추천서 발급일로부터 3개월\r\n   대출기한 내에 기업체가 기한연장을 신청할 경우 1회에 한하여 2개월 연장 가능(대출 진행 중이거나 공장시설 공사가 진행 중인 업체로 필요하다고 인정되는 경우)\r\n 9. 대출 취급은행\r\n   취급은행(10개) : 광주, 기업, 산업, 국민, 하나, 신한, 우리,  농협중앙회, 수협, 부산\r\n       기금관리은행(광주은행)과 전대 약정 미체결 은행은 취급은행에서 제외\r\n 10. 제재 및 환수 사항\r\n   지원 용도 외 자금 사용(임대수입, 용도전환, 후속계획 미이행 등) 기업은 해당자금에 대하여 전액 회수 및 향후 3년간 융자 추천 제한\r\n     거짓이나 부정한 방법으로 융자금을 (신청)지급 받은 경우\r\n     신청지원 목적과 다른 용도에 융자금을 사용한 경우\n(사업장 매매임대 등 수익 향유, 특수관계인 간 계약 등 거래, 업종 변경내역 미신고 등)\r\n   사업완료 후 1개월 이내 기업별 사업 결과보고서 제출(별지 2-1)을 의무화하고, 미제출 기업에는 당해연도와 다음연도 재신청(지원) 제한\r\n 11. 기타사항\r\n   공고 내용 이외의 사항은 전라남도 중소기업 육성기금 조례및 같은 조례 시행규칙, 2025년도 전라남도 중소기업 육성자금 지원계획에 의하며, 접수된 서류는 반환하지 않음\r\n   접수 후 심사 과정에서 필요시 추가 자료를 요청할 수 있음.\r\n   추천 및 대출 실행 후 거래은행, 상호 및 대표자 등 변경사항 발생 시 변경신청서(별지 제5호 서식)를 작성하여 전라남도중소기업일자리경제진흥원에 제출하여야 함.\r\n   신청 서식은 전남도청, 전남중소기업일자리경제진흥원 누리집에서 확인 가능 \r\n       전남도청(\x03\x00\x00\x00\x00\x03http://www.jeonnam.go.kr)\x04h\x00\x00\x00\x00\x04 : 분야별 정보  일자리경제  기업지원  자금지원\r\n       전남중소기업일자리경제진흥원(http://www.jepa.kr) : 자금지원  자금지원 공고 \r\n        전남중소기업일자리경제진흥원 자금시스템(http://\x03\x00\x00\x00\x00\x03www.jnfund.kr)\x04h\x00\x00\x00\x00\x04 : 정보광장  공지사항\r\n   온라인 접수 후 심사과정에서 필요한 추가요청자료를 1개월 이내에 보완하지 못할 시 신청이 취소됨\r\n   기타 자세한 사항은 전남중소기업일자리경제진흥원(061-288-3831~3)로 문의하시기 바람.\r\n[별지 제1호 서식] \r\n\x0b\x00\x00\x00\x00\x0b\r\n전라남도 중소유통업 구조개선자금 신청서\r\n(시설  운영)\r\n필\r\n수\r\n사\r\n항\r\n업 체 명\r\n개  업\r\n년월일\r\n대\r\n표\r\n자\r\n성    명\r\n사 업 자\r\n등록번호\r\n생년월일\r\n                 남 여\r\n법    인\r\n등록번호\r\n전    화 \r\n사업장\r\n소재지\r\n 자가  임차\r\n주소 : (      -      )  \r\n          \r\n \r\n팩    스 \r\n핸 드 폰\r\nE-mail \r\n업 종\r\n 제조 제조서비스 지식서비스 \r\n 기타(                      )\r\n주 품 목\r\n(종 목)\r\n \r\n업체구분\r\n  일 반 기 업\r\n 전 문 상 가\r\n ( 중소백화점,  쇼핑센터   전통시장)\r\n  기타(                    )\r\n담 보\r\n제 공\r\n방 법\r\n  부동산 \r\n 보증서  신용보증재단\r\n 신용보증기금 \r\n 기술보증기금\r\n 기  타 (               )\r\n대 출\r\n은 행\r\n                       은행 \r\n  (                    지점)\r\n신청액\r\n계\r\n백만원\r\n시설자금\r\n백만원\r\n목적외 사용금지\r\n  목적외 사용시 융자금을 조기 환수, 지원 제한 등 제재조치가 취해질 수 있음을 확인함\r\n운영자금\r\n백만원\r\n준\r\n필\r\n수\r\n사\r\n항\r\n종업원수\r\n(상시근로자)\r\n담 당 자\r\n연 락 처\r\n전  화 : \r\n팩  스 :            \r\n핸드폰 :\r\nE-mail : \r\n매 출 액\r\n(최근1년)\r\n                          백만원\r\n총 소요자금\r\n                       백만원\r\n지방중소기업육성자금 기대출잔액\r\n                          백만원\r\n   위와 같이 전라남도 중소유통업 구조개선자금을 신청하며 기재사항 및 붙임서류에 허위가 없음을 확인합니다.\r\n                                                 2025년       월       일\r\n                                          신청인(대표)                      (인)\r\n  전라남도지사전라남도중소기업일자리경제진흥원장 귀하\r\n[별지 제2호 서식]\r\n사  업  계  획  서\r\n 사업기간 :      년    월     -      년    월\r\n 사업개요\r\n   위치, 규모(부지, 건물연면적), 구조, 용도 등 기재\r\n 소요자금 및 조달계획주2)\r\n                                                                            (단위:백만원)\r\n\x0b\x00\x00\x00\x00\x0b\r\n구       분\r\n총 소 요 자 금\r\n유통업 구조개선자금 \r\n(신 청 자 금)\r\n기 타 자 금\r\n물량\r\n(면적/수량)\r\n금액\r\n물량\r\n(면적/수량)\r\n금액\r\n타 차입금\r\n자기자금\r\n총      계\r\n시\r\n설\r\n자\r\n금\r\n소    계\r\n전문상가/\r\n공동창고\r\n건     축\r\n기계설비\r\n점포시설개선\r\n운   영   자   금\r\n \r\n     주2) 시설자금 목적별(건축/기계설비)로 대출은행과 협의 후 신청자금 기재\r\n 융자금 상환계획\r\n  \r\n 기    타\r\n  \r\n[별지 제2-1호 서식]\r\n중소기업육성자금 사업완료 보고서\r\n1. 기업현황\r\n\x0b\x00\x00\x00\x00\x0b\r\n업 체 명\r\n대 표 자\r\n소 재 지\r\n전화번호\r\n주생산품\r\n종업원수\r\n      명(남   명, 여   명)\r\n2. 지원현황  20  년도 제  차                자금\r\n  가. 시   설\r\n\x0b\x00\x00\x00\x00\x0b\r\n시   설   명\r\n규  격\r\n수  량\r\n시공업체(전화번호)\r\n사업기간\r\n  나. 융자금\x10\x00\x00\x00\x00\x10\r\n\x0b\x00\x00\x00\x00\x0b\r\n대출은행\r\n결정 통보액\r\n대출금액\r\n대출완료일자\r\n           은행\r\n           지점\r\n   계 :         백만원\r\n  시설:         백만원\r\n  운전:         백만원\r\n   계 :        백만원\r\n  시설:        백만원\r\n  운전:        백만원\r\n \r\n3. 사업추진 중 문제점 및 건의사항\r\n\x0b\x00\x00\x00\x00\x0b\r\n  붙임 : 1. 사업추진 사진(사업별 전, 후 각 1매)\r\n       \x1f 2. 시설자금(대출금액)의 세금계산서 사본\r\n                 위와 같이 사업완료 보고서를 제출합니다.\r\n                               년         월          일\r\n             기업명 :                     대표자 :               (인)\r\n전라남도지사전라남도중소기업일자리경제진흥원장  귀하\r\n[별지 제3호 서식]\r\n대 출 상 담 확 인 서\r\n                          업체명 :\r\n                          대표자 :\r\n                          대출가능(상담)금액 :             원\r\n  위 업체는 금번 귀 도의 중소기업 육성자금을 지원받기 위하여 당행과 사전에 상담을 하였으므로, 귀 도에서 자금지원을 결정할 경우 지원 결정된 금액 이내로 당행 내규 등에 위배되지 않는 범위 내에서 대출 취급할 수 있음을 확인합니다.(단, 확인서 유효기간은 접수일 기준 1개월 이내 작성분에 한함)\r\n2025년     월     일\r\n담당자 :         은행     지점         (인)\r\n확인자 :         은행     지점장       (인)\r\n  전라남도지사전라남도중소기업일자리경제진흥원장 귀하 \r\n[별지 제4호 서식]\r\n\x0b\x00\x00\x00\x00\x0b\r\n기업정보의 제공활용 및 이용 동의서\r\n전라남도지사전라남도중소기업일자리경제진흥원장 귀하\r\n 본인은 전라남도 중소유통업구조개선자금 신청서및 제반서류(사업계획서, 기타 자금 융자에 필요한 서류 일체)에 기재된 기업신용정보를 해당 시군 및 공공기관의 정책 자료로 활용하도록 하는 것에 대해 동의합니다.\r\n 개인정보보호법에 따라, 전라남도 중소기업 육성기금 조례등에 따른 지원업체로 결정될 경우 당사의 정보(업체명, 대표자명, 사업자번호, 지원결정일, 지원사업명, 지원금액 등)를 중소기업 지원사업 통합관리시스템에 공개하는 것에 동의합니다.\r\n\x0b\x00\x00\x00\x00\x0b\r\n 중소기업 지원사업 통합관리시스템에서 사업 참여(신청 및 지원) 기업정보의 수집조회 및 활용\r\n 중소기업기본법 제20조의2에 따라 중소기업 지원사업에 신청한 기업의 과세정보(국세기본법 제81조의13 및 관세법 제116조), 고용정보(고용보험법 제2조의1), 직장가입자 수(국민건강보험법 제6조) 등\r\n 기업정보는 신청 및 지원기업의 사업 신청 이전 3개년부터 참여 이후 5년간 수집\r\n 중소기업 지원사업 통합관리시스템에 대한 동의서 징구는 민법 제114조의 행정행위의 대리권 행사방식의 현명주의를 적용하여, 중소기업 지원사업의 효율적인 운영을 위해 본 사업의 소관부처 및 수행기관이 대신하여 받는 것임을 알려 드립니다.\r\n 전라남도에 자금을 융자받기 위해 제출하는 제반서류(신청서, 사업계획서, 기타 자금 융자에 필요한 서류 일체) 제출 시 사실과 다르거나 허위의 자료를 제출 하여 융자를 신청한 경우에는 자금지원 결정취소, 조기상환, 신규참여제한 등의 제재 또는 법적조치가 취해질 수 있다는 것에 동의합니다.\r\n2025년   월   일\r\n                               기 업 명 : \r\n                               대 표 자 :                           (인)\r\n* 본인은 위 각 정보의 수집조회활용에 관한 자세한 설명을 듣고, 본 동의서 내용을 충분히 이해하고 동의하였습니다.\r\n[별지 제5호 서식]  자금추천서 발급 후 해당기업만 제출(대출취급일자 연장 또는 은행변경 시  작성\r\n전라남도 중소기업소상공인 육성자금 변경 신청(신고)서\r\n\x0b\x00\x00\x00\x00\x0b\r\n지원자금명\r\n         년  제(      )차 (                      ) 자 금\r\n업  체  명\r\n(사업자등록번호)\r\n(        -     -          )\r\n대표자명\r\n(생년월일)\r\n(                 )\r\n소 재 지\r\n연 락 처\r\n전화)\r\n팩스)\r\nH P)\r\n변 경 사 유\r\n(상세하게 기재)\r\n 은행변경 및 대출기한 연장 등 해당기업만 작성 제출\r\n변    경    전\r\n변     경     후\r\n \r\n 상기와 같이 전라남도 중소기업 육성자금을 변경 신청합니다.\r\n \t\x00\x00\x00\x00\t\t\x00\x00\x00\x00\t\t\x00\x00\x00\x00\t\t\x00\x00\x00\x00\t         2025년\t\x00\x00\x00\x00\t   월        일\r\n   \t\x00\x00\x00\x00\t\t\x00\x00\x00\x00\t\t\x00\x00\x00\x00\t\t\x00\x00\x00\x00\t  신청인                       \x1f\x1f\x1f\x1f\x1f\x1f\x1f (인)\r\n   전라남도지사  전라남도중소기업일자리경제진흥원장 귀하\r\n\x1f   \r\n    붙임서류  \r\n    은행(지점) 또는 대출취급기한 변경시 - 육성자금추천결정통보서 사본 1부 \r\n    대표자 또는 주소변경시 - 사업자등록증 및 법인등기부등본(말소사항포함) 각 1부\r\n신고대상 : 대표자 변경사항, 상호소재지의 변경, 휴폐업, 담보 종류의 변경, 기타 업체의 변동사항\r\n승인대상 : 법인 전환, 주생산업종의 변경, 계약대상시설의 변경, 추천기한취급은행 변경\r\n    문의 : 061)288-3832 / 접수 : \x03\x00\x00\x00\x00\x03www.jnfund.kr\x04h\x00\x00\x00\x00\x04\r\n\n'
        """},
        {'role' : 'assistant', 'content' : """
        📌 '[전남] 2025년 중소기업 육성자금(중소유통업 구조개선자금) 지원계획 공고'
         
        🤖 사업 개요
        - 「광주광역시 중소기업육성기금 특별회계설치 및 운용조례」 제7조의 규정에 따라 2025년도 광주광역시  중소유통업구조개선자금 지원계획을 다음과 같이 공고합니다.

        ✅ 사업자 유형
        - 법인사업자
        - 개인사업자
         
        ☑️ 지원업종
        - 도매 및 상품 중개업
        - 소매업(자동차 제외)

        💡 공고특성
        - 소상공인
        - 대출
        - 보조금
        - 시설·환경 개선
         
        👥 지원대상
        - 지원 신청일 현재 전라남도 내에 사업자등록을 한 업체
        - 한국표준산업분류 중 도소매업 및 상품중개업자
        - 프랜차이즈 본부 및 가맹점(유통업에 한함)
        - 전통시장, 중소백화점쇼핑센터 동의 개설자 또는 관리자

        📅 신청기간
        반기별 접수(공고일 ~, 6. 1. ~)

        📝 신청방법
        - 온라인 접수 
        - 전남중소기업일자리경제진흥원 자금시스템(www.jnfund.kr) 
        ㅇ 문 의 처 : 전남중소기업일자리경제진흥원 기업육성팀 (061-288-3832~4)
         
        🚫 지원제외 대상
        - 국세지방세 체납중인 기업
        - 휴폐업중인 업체
        - 전라남도 중소기업 육성자금(시설자금) 이용 중이거나, 당해연도 추천 후 대출하지 않은 기업(단, 이용한도가 남은 기업은 한도 내 이용 가능)
        - 당해 연도 정부 및 공공기관으로부터 시설자금 수혜기업
        - 중소벤처기업부 중소기업지원사업 통합관리시스템에서 접수일 기준 최근 5년간 정부기관 및 지자체 등의 지원금액(사업분야: 금융)과 본 신청 건을 포함하여 지원금액이 100억원을 초과하는 기업(금융 보증/보험 제외)
        - 기타 자금지원 관련 규정에서 제외하도록 정한 업체

        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X
        
        ⚠ 유의사항
        - 지원 용도 외 자금 사용(임대수입, 용도전환, 후속계획 미이행 등) 기업은 해당자금에 대하여 전액 회수 및 향후 3년간 융자 추천 제한
        - 거짓이나 부정한 방법으로 융자금을 (신청)지급 받은 경우
        - 신청지원 목적과 다른 용도에 융자금을 사용한 경우(사업장 매매임대 등 수익 향유, 특수관계인 간 계약 등 거래, 업종 변경내역 미신고 등)
        - 사업완료 후 1개월 이내 기업별 사업 결과보고서 제출(별지 2-1)을 의무화하고, 미제출 기업에는 당해연도와 다음연도 재신청(지원) 제한
         
        📌 선정절차
        - 전라남도 중소기업 육성기금 조례 시행규칙제13조의 규정에 따라 제외 대상업체 및 결격 여부를 심사하여 융자대상 업체 선정
         
        💰 지원내용
        - 지원규모 : 30억원
        | 구분 | 지원사업 | 융자금액 | 융자기간 | 융자이율 |
        |------|----------|----------|----------|----------|
        | 시설 투자 | 전문상가 건립, 전문상가 시설개선, 공동창고 건립 | 4억원 이내 | 3년거치 5년 분할상환 | 연3.0% (25.1분기 기준) - 정부정책자금 기준금리에 따라 분기별 변동 금리 적용 |
        | 점포시설개선 | | 1억원 이내 | | |
        | 운영 자금 | 경쟁력강화 및 경영활성화 자금 | 1억원 이내 | 1년거치 2년 분할상환 | |
        <연속지원 제한>
        - 시설자금 상환 후 2년 이내, 운영자금 상환 후 1년 이내 융자 지원 제한
         
        📞 문의처
        061-288-3832~4 (기업육성팀)
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : """
        당신은 정부 지원 사업 전문 분석가입니다. 
        사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
        요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

        사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
        다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

        신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

        지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
        신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

        신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
        제공하는 템플릿 양식 : {base_template}
        단어 정의 사전 : {keyword_dictionary}
        참고 : {explanation}

        각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
        """},
        {'role' : 'user', 'content' : """
        
        """},
        {'role' : 'assistant', 'content' : """
        📌 2025년 상반기 보증연계투자 대상기업 공모 요약 안내
         
        🤖 사업 개요
        - 기술보증기금(이하 ‘기보’)에서는 기술력과 사업성이 우수한 유망 창업기업에 대한 투자 활성화를 위해 다음과 같이 2025년 상반기 보증연계투자 대상기업을 공모하니, 많은 관심과 참여를 바랍니다.

        ✅ 사업자 유형
        - 법인사업자
        - 개인사업자
         
        ☑️ 지원업종
        - 제조업, 정보통신업, 전문, 과학 및 기술 서비스업

        💡 공고특성
        -
         
        👥 지원대상
        기술력, 사업성 및 성장잠재력 등을 갖춘 중소기업으로, 아래 기본요건을 모두 충족하는 창업기업
        - 법인설립 후 5년 이내의 기술혁신선도형기업
        * 단, R&D 및 녹색혁신성장미래전략산업 영위기업 등은 설립 후 5년 초과 7년 이내 기업도 가능
        - 기보 보증기업 또는 보증승인(예정)기업
        - 최근 1년 이내 기보 기술사업평가등급이 BB등급 이상인 기업
        * 최근 1년 이내 평가등급이 없더라도 기보 영업점 담당자가 투자지원 가능성이 높다고 판단한 기업은 신청 가능
        - 보증금지기업 또는 보증제한기업에 해당되지 않는 기업
        - 벤처캐피탈, 은행 등 기관투자자로부터 투자를 받지 않은 기업
        * 단, 지방소재기업 및 모태출자조합으로부터 3억원 이하 투자를 받은 기업은 신청 가능
        - 보증연계투자(투자옵션부보증 포함) 잔액이 없는 기업

        📅 신청기간
        25. 3. 17 ∼ 25. 5. 30 17:00까지

        📝 신청방법
        - 기보 영업점 담당자와 상담 후 접수
         
        🚫 지원제외 대상
        -
         
        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X 
         
        💁 청년 대상
        X 
         
        ⚠ 유의사항
        - 
         
        📌 선정절차
        - 
         
        💰 지원내용
        - 
        
        📞 문의처
        - 
         
        """}
    ]},
]


In [69]:
from collections import defaultdict # 딕셔너리 디폴트 값을 지정해주는 모듈
import numpy as np
import time
import json
import tiktoken # openai 모델들이 사용하는 base 토큰화+인코등 모듈

## 1. 데이터셋의 형식 준수 여부 검증

In [71]:
# defaultdict : 딕셔너리를 만들면서 존재하지 않는 key에 대한 디폴트값 설정을 제공
#               (int로 설정할 경우, 존재하지 않는 key가 들어오면 정수형 기본값이 0으로 동작함)
format_errors = defaultdict(int)

for ex in dataset : # ex는 하나의 대화(messages)로 dict 형태!
    # 데이터 셋의 각 대화들이 dict 형태가 아니라면
    # isinstance : 어떤 클래스의 객체인지 확인하는 함수(객체명, 클래스명)로 T, F로 출력
    if not isinstance(ex, dict) : 
        # dict 형태가 아니라면
        format_errors['data_type'] += 1 # format_errors에는 {'data_type':0}인 상태가 디폴트로 생성
        continue

    # messages의 value값이 있는지 없는지 판단
    messages = ex.get('messages', None)
    # messages가 False라면 조건식이 True(messages의 value값이 없다면 None이 될 것이고 파이썬은 이를 F로 인식)
    if not messages :
        format_errors['missing_messages_list'] += 1
        continue

    for message in messages :
        # 모든 message들의 key값을 확인하고 아래 튜플 내 값들 중 하나로 인식되지 않는다면!
        # (채팅 API가 아닌 경우 role, content 외에 function_call과 같은 다른 값들이 있을 수 있음)
            # any : 괄호 안에 조건 중 하나라도 T인 경우 T를 반환
            # any를 사용하지 않았다면 조건식을 or로 매우 길게 작성해야 함
        if any(key not in ('role', 'content', 'name', 'function_call', 'weight') for key in message) :
            format_errors['message_unrecognized_key'] += 1
        # 요약하면 message내의 key 값이 튜플 내의 값들 중 하나가 아니라면 카운트를 +1 시켜주는 코드

        # message에 role과 content가 있는지 다시 확인
        if 'role' not in message or 'content' not in message :
            format_errors['message_missing_key'] += 1

        # message의 role이 system, user, assistant 중 하나가 아니라면
            # message는 dict 형태이므로 value값을 판단하기 위해서는 key인 role로 접근해야함
        if message.get('role', None) not in ('system', 'user', 'assistant') :
            format_errors['unrecognized_role'] += 1

        # content에 어떤 value값이 있는지 확인
        content = message.get('content', None)
        # content에 값이 존재하지 않거나 문자열이 아니라면 
        if not content or not isinstance(content, str) :
            format_errors['missing_content'] += 1

    # 각 대화에 assistant의 응답이 하나도 없다면
    # messages들 중 하나라도 assistant의 응답이 있으면 any문은 전체를 T로 인식함
    if not any(message.get('role', None) == 'assistant' for message in messages) :
        format_errors['example_missing_assistant_message'] += 1

# 발생한 에러가 있다면(format_errors에 값이 있다면) 에러 내용을 출력
if format_errors :
    print('에러 발견 :')
    for key, value in format_errors.items() :
        print(f"{key}: {value}")
else :
    print('에러가 없습니다 ^^')

에러가 없습니다 ^^


## 2. 누락 메시지 식별 및 메시지와 토큰 수 확인

In [72]:
# get_encoding : 특정 토크나이저 인코딩 모델 검색
# cl100k_base : OpenAI 최신 LLM에 사용되는 모델명
encoding = tiktoken.get_encoding('cl100k_base')

# 메시지 목록의 총 토큰 수 계산
    # token_per_message : 메시지 내 'role', 'content, '{}' 요소들에 대한 토큰 개수(한 요소당 1로 대략 계산)
    # tokens_per_name : key 값으로 'name'이 존재할 때 토큰 1개 추가(사용자가 assistant의 이름을 지정할 경우 발생)
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1) :
    num_tokens = 0
    for message in messages :
        num_tokens += tokens_per_message
        for key, value in message.items() :
            # 인코딩된 값을 토큰으로 씀
            # value값(role 및 content의 value)이 tiktoken에 의해 토큰화+인코딩 된 길이(토큰 수)를 추가해줌
            num_tokens += len(encoding.encode(value))
            # key에 name이 있다면(assistant의 이름을 설정하면 name이 자동 생성됨)
            if key == 'name' :
                num_tokens += tokens_per_name

    # num_tokens를 3씩 증가시키는 이유 : 
        # 1) 하나의 messages 전체를 감사는 토큰 : "{}"
        # 2) 한 message의 끝을 표시하는 특수 토큰(모델 내부에서 동작, <SEP> 등)
        # 3) 각각의 message를 구분해주는 구분 토큰 : ','
    num_tokens += 3
    return num_tokens

# assistant(모델)가 응답한 메시지의 총 토큰 수(모델의 max_output_tokens 기준에 맞는지 판단 가능)
def num_assistant_tokens_frome_messages(messages) :
    num_tokens = 0 
    for message in messages :
        if message['role'] == 'assistant' :
            # content의 value값만 토큰화+인코딩
            num_tokens += len(encoding.encode(message['content']))
    return num_tokens

# message의 토큰 길이에 대한 통계 정보 출력
def print_statistics(values) :
    print(f'min / max : {min(values)},{max(values)}')
    print(f'mean / median : {np.mean(values)}, {np.median(values)}')

In [70]:
# 코드 및 함수 실행
max_output_tokens = 16384   # 모델 최대 출력 토큰 수(gpt-4o-mini의 경우 16384개)
n_missing_system = 0        # role에 system이 없는 경우의 대화 수 카운트
n_missing_user = 0          # role에 user가 없는 경우의 대화 수 카운트
n_messages = []             # 각 대화의 message 개수가 담길 빈 리스트
total_tokens_lens = []      # 각 대화의총 토큰 수가 담길 빈 리스트
assistant_message_lens = [] # assistant가 보낸 메시지의 길이가 담길 빈 리스트

for ex in dataset :
    messages = ex['messages']
    # 각 message에서 role이 system인 경우가 하나라도 있다면 T이지만 앞에 not이 붙어서 Frk ehla
    if not any(message['role']=='system' for message in messages) :
        n_missing_system += 1
    if not any(message['role']=='user' for message in messages) :
        n_missing_user += 1

    # 각 messages의 message 개수를 리스트에 추가
    n_messages.append(len(messages))
    # 메시지 목록의 총 토큰 수 함수 실행 후 리스트에 추가
    total_tokens_lens.append(num_tokens_from_messages(messages))
    # 모델 출력 토큰 수 함수 실행 수 리스트에 추가
    assistant_message_lens.append(num_assistant_tokens_frome_messages(messages))

print('System 메시지 누락 수 :', n_missing_system)
print('User 메시지 누락 수 :', n_missing_user)

print()
print("### 대화 당 메시지 수 통계 : ")
print_statistics(n_messages)

print()
print("### 대화 당 전체 토큰 수 통계 : ")
print_statistics(total_tokens_lens)

print()
print("### 대화 당 assistant 수 통계 : ")
print_statistics(assistant_message_lens)

print()
# total_tokens_lens(각 메시지들의 총 토큰 수 리스트)에서 각각의 값을 16384와 비교하여 넘어가는지 판단
    # 파이썬에서 True는 1, False는 0이므로 이를 sum한다면 True값의 개수를 파악할 수 있음
n_too_long = sum(i > max_output_tokens for i in total_tokens_lens)
print(f"\n{n_too_long}개의 대화가 {max_output_tokens}개 토큰 제한을 초과하며 이 부분은 학습 중 잘릴 수 있습니다.")

System 메시지 누락 수 : 0
User 메시지 누락 수 : 0

### 대화 당 메시지 수 통계 : 
min / max : 3,3
mean / median : 3.0, 3.0

### 대화 당 전체 토큰 수 통계 : 
min / max : 1568,13934
mean / median : 5975.6, 3252.0

### 대화 당 assistant 수 통계 : 
min / max : 891,3326
mean / median : 1699.6, 1490.0


0개의 대화가 16384개 토큰 제한을 초과하며 이 부분은 학습중 잘릴 수 있습니다.


In [96]:
# 모델에 맞는 대화당 최대 토큰 수 설정
MAX_TOKENS_PER_EXAMPLE = 65536

# 학습 횟수 설정
TARGET_EPOCHS = 5
MIN_DEFAULT_EPOCHS = 1 
MAX_DEFAULT_EPOCHS = 1
MIN_TARGET_EXAMPLES = 5
MAX_TARGET_EXAMPLES = 100

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)

# 대화의 개수와 TARGET_EPOCHS의 곱이 최소 데이터 수 보다 작다면
if(n_train_examples * TARGET_EPOCHS) < MIN_TARGET_EXAMPLES:
    # OpenAI 테스트 기준 epochs 최대치(25회)와 최소 데이터(100개)에서 우리 데이터 수를 나눈 몫을 비교하여
    # 더 낮은 값으로 최종 학습 횟수(n_epochs) 설정
    n_epochs = min(MAX_DEFAULT_EPOCHS, (MIN_TARGET_EXAMPLES // n_train_examples))

elif (n_train_examples * TARGET_EPOCHS) > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, (MAX_TARGET_EXAMPLES // n_train_examples))
    
## 대화별 총 토큰 수가 담긴 total_tokens_lens 리스트에서 값을 가져와 65536과 비교하여
## 낮은 값으로 합해서 총 토큰 수를 계산함(총 토큰 수가 65536보다 놏다면 65536개까지만 제한하는 식)
## 실제로 더 긴 토큰이 들어가도 모델의 최대치인 65536개 까지만 반영될 것이기 때문
n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in total_tokens_lens)

print(f'데이터 셋에는 학습 중 요금이 청구될 {n_billing_tokens_in_dataset}개의 토큰이 있습니다.')
print(f'기본적으로 이 데이터 셋에서 {n_epochs} epoch 동안 학습합니다.')
print(f'총 {n_epochs*n_billing_tokens_in_dataset}개의 토큰에 대해 요금이 청구 됩니다.')

데이터 셋에는 학습 중 요금이 청구될 29878개의 토큰이 있습니다.
기본적으로 이 데이터 셋에서 5 epoch 동안 학습합니다.
총 149390개의 토큰에 대해 요금이 청구 됩니다.


In [ ]:
with open("finetune_data/bizinfo.dataset.jsonl", "r", encoding='utf-8') as f :
    dataset = [json.loads(line) for line in f]

print("샘플 수 :", len(dataset))
dataset

In [91]:
# fine-tuning을 위한 파일 생성 객체 만들기
fine_tune_files = client.files.create(
    file = open("finetune_data/bizinfo.dataset.jsonl", "rb"),
    purpose = "fine-tune"
)

fine_tune_files

FileObject(id='file-EcY3ArUTWFPGx158dEVRmx', bytes=134725, created_at=1742871506, filename='bizinfo.dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [107]:
# 작업 객체 생성
fine_tune_job = client.fine_tuning.jobs.create(
    model = 'gpt-4o-mini-2024-07-18',
    # fine_tune_files(데이터 파일 객체)의 id값을 지정해줘야 함
    training_file = fine_tune_files.id
)

fine_tune_job

FineTuningJob(id='ftjob-j0hunaBbBQd2B2y3SOHNjUC8', created_at=1742872919, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-lGTjgeNkAw28xIQ1qTU7ATOs', result_files=[], seed=386540105, status='validating_files', trained_tokens=None, training_file='file-EcY3ArUTWFPGx158dEVRmx', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None, metadata=None)

In [98]:
client.fine_tuning.jobs.retrieve("ftjob-vYpPYNlGkuuticxtDnnayWRD")

FineTuningJob(id='ftjob-vYpPYNlGkuuticxtDnnayWRD', created_at=1742871538, error=Error(code='invalid_n_examples', message='Training file has 5 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-lGTjgeNkAw28xIQ1qTU7ATOs', result_files=[], seed=1554921518, status='failed', trained_tokens=None, training_file='file-EcY3ArUTWFPGx158dEVRmx', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=20)), type='supervised'), user_provided_suffix=None, metadata=None)

In [ ]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id='ftjob-vYpPYNlGkuuticxtDnnayWRD',
                                    limit=3)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-4SMGXpSFKFN6QP1lVd5oaH0E', created_at=1742871570, level='error', message='Training file has 5 example(s), but must have at least 10 examples', object='fine_tuning.job.event', data={'error_code': 'invalid_n_examples', 'error_param': 'training_file'}, type='message'), FineTuningJobEvent(id='ftevent-mZc9GD8hkkm01t116Wi0k6x6', created_at=1742871538, level='info', message='Validating training file: file-EcY3ArUTWFPGx158dEVRmx', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-NvJpuBp1Q19IiFqmbF33nsTF', created_at=1742871538, level='info', message='Created fine-tuning job: ftjob-vYpPYNlGkuuticxtDnnayWRD', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [58]:
# fine tuning template

    # {'message': [
    #     {'role' : 'system', 'content' : """
    #     당신은 정부 지원 사업 전문 분석가입니다. 
    #     사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
    #     요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

    #     사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
    #     다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

    #     신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

    #     지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
    #     신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

    #     신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
    #     제공하는 템플릿 양식 : {base_template}
    #     단어 정의 사전 : {keyword_dictionary}
    #     참고 : {explanation}

    #     각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
    #     """},
    #     {'role' : 'user', 'content' : ''},
    #     {'role' : 'assistant', 'content' : """
    #     📌 
         
    #     🤖 사업 개요
    #     - 

    #     ✅ 사업자 유형

         
    #     ☑️ 지원업종


    #     💡 공고특성

         
    #     👥 지원대상


    #     📅 신청기간


    #     📝 신청방법
    #     - 
         
    #     🚫 지원제외 대상
    #     -
         
    #     ⭐️ 우대사항
    #     - 
         
    #     👩 여성 대상
    #     - 
         
    #     💁 청년 대상
    #     - 
         
    #     ⚠ 유의사항
    #     - 
         
    #     📌 선정절차
    #     - 
         
    #     💰 지원내용

    #     📞 문의처
    #     """}
    # ]},

In [40]:
import streamlit as st
import json
import re

content = """
1인 자영업자 사회보험료 지원사업 제출서류
        | 구분          | 제출서류                                                                 | 비고                                                                 |
        |---------------|--------------------------------------------------------------------------|----------------------------------------------------------------------|
        | 필수 제출서류 | ① 사업신청서(행정정보공동이용동의서 포함) [서식1]                      |                                                                      |
        |               | ② 중소기업통합관리시스템 기업정보수집·이용·제공동의서 [서식4]          |                                                                      |
        |               | ③ 통장사본(스마트폰 캡처본도 인정)                                      | 수수료 없음                                                          |
        | 선택 제출서류 | ① 사업자등록증 또는 사업자등록증명원 사본 1부                          |                                                                      |
        |               | ② 건강보험자격득실확인서 사본 1부 (신청일로부터 1개월 이내 발급본)     | * 국민건강보험공단 홈페이지(www.nhis.go.kr) 또는 정부24에서 발급 가능 |
        |               |                                                                          | 행정정보공동이용 미동의 시 별도 제출
"""

# 마크다운 표 감지 (| 로 시작하는 줄이 연속되면 표로 간주)
markdown_table_pattern = r"(\n\|.*\|\n\|[-:]+\|(?:\n\|.*\|)*)"
matches = re.findall(markdown_table_pattern, content)

if matches:
    # 텍스트와 표를 분리
    text_parts = content.split(matches[0])[0]  # 표 앞의 텍스트 부분
    table_part = matches[0]  # 마크다운 표 부분

    # 텍스트 출력
    if text_parts.strip():
        st.markdown(text_parts)  # 제목, 설명 등

    # 표 출력
    st.markdown(table_part)  # 마크다운 표 그대로 렌더링

else:
    # 표가 없으면 그냥 전체 텍스트 출력
    st.markdown(content)


2025-03-20 17:22:33.625 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 17:22:33.736 
  command:

    streamlit run /opt/homebrew/anaconda3/envs/study/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-20 17:22:33.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
